In [ ]:
import pandas as pd
import glob
import numpy as np
import pprint
import pickle
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from feature_engine.selection import DropDuplicateFeatures, DropConstantFeatures

## Create a list of the Cancer Data folders

In [ ]:
# create a list of folder names that will be used to look for the files
folders = ['TCGA-ACC', 'TCGA-BLCA', 'TCGA-BRCA', 'TCGA-CESC', 'TCGA-CHOL', 'TCGA-COAD', 'TCGA-DLBC',
 'TCGA-ESCA', 'TCGA-GBM', 'TCGA-HNSC', 'TCGA-KICH', 'TCGA-KIRC', 'TCGA-KIRP', 'TCGA-LAML', 'TCGA-LGG',
  'TCGA-LIHC', 'TCGA-LUAD', 'TCGA-LUSC', 'TCGA-MESO', 'TCGA-OV', 'TCGA-PAAD', 'TCGA-PCPG', 'TCGA-PRAD',
   'TCGA-READ', 'TCGA-SARC', 'TCGA-SKCM', 'TCGA-STAD', 'TCGA-TGCT', 'TCGA-THCA', 'TCGA-THYM', 'TCGA-UCEC',
    'TCGA-UCS', 'TCGA-UVM']

In [ ]:
# Index values that will be used for each cancer type
# change this to process a specific cancer type (0-32)
Indx = 0

##  miRNA data preprocessing

### 1. Load Files

In [ ]:
# Load data from the subdirectories of [parent_directory]
# The pattern is: *.mirna.tsv

# Load the data for the first element of folders list
mirna_list = glob.glob("[parent_directory]**/{}.mirna.tsv".format(folders[Indx]), recursive=True) # change [parent_directory] to the parent directory where the data is stored

# sort the list of files
mirna_list.sort()
# Load the file
if len(mirna_list) > 0:
    for file in mirna_list:
        print("Loading file: " + file)
        df = pd.read_table(file)
        print(df.shape)
else:
    print("No file matching the pattern found.")
print(len(mirna_list))

In [ ]:
# load the first file and print the shape
mirnafile1 = mirna_list[0]
stemmirna = pd.read_table(mirnafile1)
print(mirnafile1)
print(stemmirna.shape)

### 2. Transpose Data

In [ ]:
mirnastem=stemmirna.transpose()
print(mirnastem.shape)
mirnastem=mirnastem.reset_index()
print(mirnastem.shape)
mirnastem.columns = mirnastem.iloc[0]
#remove first row from DataFrame
mirnastem = mirnastem[1:]
mirnastem=mirnastem.rename(columns={f'{mirnastem.columns[0]}':'sample'})
mirnastem

### 3. Drop Constant Features (i.e. >99.8% similarity)

In [ ]:
from feature_engine.selection import DropConstantFeatures
sel1 = DropConstantFeatures(tol=0.998, variables=None, missing_values='raise')
sel1.fit(mirnastem)
mirnastem = sel1.transform(mirnastem)
mirnastem.shape
mirnastem

### 4. Remove Colinear Features (i.e. >80% correlation)

In [ ]:
# temporary remove the first column for later adding it to the first column
mirnastem1 = mirnastem.iloc[:, 1:]
# check the variable format with pandas dtypes.
print(mirnastem1.dtypes)
# convert the variable to numerical variables
mirnastem1 = mirnastem1.astype(float)
# check the variable format with pandas dtypes.
print(mirnastem1.dtypes)
mirnastem1

In [ ]:
# remove correlated features
from feature_engine.selection import SmartCorrelatedSelection
sel2 = SmartCorrelatedSelection(
    variables=None,
    method="pearson",
    threshold=0.8,
    missing_values="raise",
    selection_method="variance",
    estimator=None,
)


In [ ]:
sel2.fit(mirnastem1)
stemmirna = sel2.transform(mirnastem1)
stemmirna.shape

In [ ]:
# add the 'sample' column from mirnastem to the first column of stemmirna
stemmirna.insert(0, 'sample', mirnastem['sample'])

In [ ]:
stemmirna

##  DNA Methylation data preprocessing

### 1. Load Files

In [ ]:
# Load data from the subdirectories of [parent_directory]
# The pattern is: *.methylation450.tsv

# Load the data
meth_list = glob.glob("[parent_directory]/**/{}.methylation450.tsv".format(folders[Indx]), recursive=True) # change [parent_directory] to the parent directory where the data is stored
# sort the list of files
meth_list.sort()
# Load the file
if len(meth_list) > 0:
    for file in meth_list:
        print("Loading file: " + file)
        df = pd.read_table(file)
        print(df.shape)
else:
    print("No file matching the pattern found.")
print(len(meth_list))

In [ ]:
# load the first file and print the shape
meth1 = meth_list[0]
dnamethylaton = pd.read_table(meth1)
print(meth1)
print(dnamethylaton.shape)

### 2. Transpose Data

In [ ]:
dnameth=dnamethylaton.transpose()
print(dnameth.shape)
dnameth=dnameth.reset_index()
dnameth.columns=dnameth.iloc[0]
dnameth=dnameth[1:]
# rename the first column to 'sample'
dnameth=dnameth.rename(columns={f'{dnameth.columns[0]}':'sample'})
dnameth.head()

### 3. Remove NANs

In [ ]:
# remove NaN values
dnameth = dnameth.dropna(axis=1)
dnameth.shape
dnameth

### 4. Drop Constant Features (i.e. >95% similarity)

In [ ]:
from feature_engine.selection import DropConstantFeatures
sel1 = DropConstantFeatures(tol=0.95, variables=None, missing_values='raise')
sel1.fit(dnameth)
dnameth = sel1.transform(dnameth)
dnameth.shape
dnameth

### 5. Remove Colinear Features (i.e. >80% correlation)

In [ ]:
dnameth=dnameth.dropna(axis=1)
print(dnameth)
dnamethyl=dnameth.drop(columns='sample')
dnameth_columns = dnamethyl.columns.tolist()
numerical_features = [col for col in dnameth_columns if dnameth[col].dtype == 'object']

from sklearn.feature_selection import VarianceThreshold 
# function that takes data and returns it after removing the features
# having less than the given threshold variance

def variance_threshold_selector(data, threshold=0.9):
    selector = VarianceThreshold(threshold)
    selector.fit(data)
    return data[data.columns[selector.get_support(indices=True)]]

hvdnameth = variance_threshold_selector(dnamethyl, 0.061) # change this value to see the effect, we have changed it so the reduced feature size is of similiar dimension.
print(type(hvdnameth))
print(hvdnameth.shape)
samplenames=dnameth['sample']
hvdnameth.insert(0,'sample',samplenames)

In [ ]:
hvdnameth

##  Gene Expression data preprocessing

### 1. Load Files

In [ ]:
# Load data from the subdirectories of [parent_directory]
# The pattern is: *.gene-expr-RNAhtseq*.tsv

# Load the data
gene_list = glob.glob("[parent_directory]**/{}.gene-expr-RNAhtseq*.tsv".format(folders[Indx]), recursive=True) # change [parent_directory] to the parent directory where the data is stored
# sort the list of files
gene_list.sort()
# Load the file
if len(gene_list) > 0:
    for file in gene_list:
        print("Loading file: " + file)
        df = pd.read_table(file)
        print(df.shape)
else:
    print("No file matching the pattern found.")
print(len(gene_list))

In [ ]:
# load the first file and print the shape
gene_file1 = gene_list[0]
genexpr = pd.read_table(gene_file1)
print(gene_file1)
print(genexpr.shape)

In [ ]:
genexpr.head(100)

### 2. Transpose Data

In [ ]:
genexpress=genexpr.transpose()
print(genexpress.shape)
genexpress=genexpress.reset_index()
print(genexpress.shape)
genexpress.columns = genexpress.iloc[0]
#remove first row from DataFrame
genexpress = genexpress[1:]
genexpress=genexpress.rename(columns={f'{genexpress.columns[0]}':'sample'})
genexpress

### 3. Drop Constant Features (i.e. >95% similarity)

In [ ]:
from feature_engine.selection import DropConstantFeatures
sel1 = DropConstantFeatures(tol=0.95, variables=None, missing_values='raise')
sel1.fit(genexpress)
genexpress = sel1.transform(genexpress)
genexpress.shape
genexpress

### 4. Remove duplicate features

In [ ]:
# set up the selector
sel2 = DropDuplicateFeatures(variables=None, missing_values='raise')
# find the duplicate features, this might take a while
sel2.fit(genexpress)

In [ ]:
# let's explore our list of duplicated features

len(sel2.features_to_drop_)

In [ ]:
# remove the duplicated features
genexpress = sel2.transform(genexpress)
genexpress.shape

### 5. Remove Colinear Features

In [ ]:
genexpress=genexpress.dropna(axis=1)
print(genexpress)
genes=genexpress.drop(columns='sample')
genes_columns = genes.columns.tolist()
numerical_features = [col for col in genes_columns if genexpress[col].dtype == 'object']

from sklearn.feature_selection import VarianceThreshold 
# function that takes data and returns it after removing the features
# having less than the given threshold variance

def variance_threshold_selector(data, threshold=0.9):
    selector = VarianceThreshold(threshold)
    selector.fit(data)
    return data[data.columns[selector.get_support(indices=True)]]

hvgenes = variance_threshold_selector(genes, 0.035)
print(type(hvgenes))
print(hvgenes.shape)
samplenames=genexpress['sample']
hvgenes.insert(0,'sample',samplenames)

In [ ]:
hvgenes

### 6. Remove genes with expression < 7

In [ ]:
# calculate column means except for the first column
hvgenes_colms=hvgenes.iloc[:,1:]
means=hvgenes_colms.mean()
print(means)
high=means[means>=10]
print(high)
print(high.index)

In [ ]:
hvgenes_colms

In [ ]:
hvgcolumns=hvgenes_colms.columns.tolist()
print(hvgcolumns)
print(len(hvgcolumns))
print(len(hvgenes_colms))

In [ ]:
genestouse=[]
for i in range(len(hvgcolumns)): 
    sc=hvgenes[f'{hvgcolumns[i]}']>7
    b=sc.value_counts()
    if len(b.index)==2:
       genestouse.append(f'{hvgcolumns[i]}')
print(genestouse)
print(len(genestouse))

In [ ]:
genedataframe=hvgenes[genestouse]
genedataframe.head()

In [ ]:
samplenames=hvgenes['sample']
genedataframe.insert(0,'sample',samplenames)

In [ ]:
genedataframe

## Survival Data

In [ ]:
# Get a list of all files matching the pattern from the subdirectories of [parent_directory]
# The pattern is: *.survival.tsv

# surv_list = glob.glob("/home/80024223/data/Xena-GDC/**/*.survival.tsv", recursive=True)
surv_list = glob.glob(" [parent_directory]**/{}.survival.tsv".format(folders[Indx]), recursive=True) # change [parent_directory] to the parent directory where the data is stored
# sort the list of files
surv_list.sort()

# Load the file
if len(surv_list) > 0:
    # print the filename and corresponding number of rows
    for file in surv_list:
        surv_df = pd.read_csv(file, sep='\t')

        print(f"{file} & {surv_df.shape}")
else:
    print("No file matching the pattern found.")
print(len(surv_list))

In [ ]:
# load the first file and print the shape
survfile1 = surv_list[0]
surv = pd.read_table(survfile1)
print(survfile1)
print(surv.shape)

In [ ]:
surv

## Clinical Data (age, gender, race, cancer stage)

### 1. Load Files

In [ ]:
# Index values that will be used for each cancer type
# change this to process a specific cancer type (0-32)
Indx = 0

In [ ]:
# Get a list of all files matching the pattern from the subdirectories of [parent_directory]
# The pattern is: *.GDC_phenotype.tsv
clin_list = glob.glob("[parent_directory]/{}.GDC_phenotype.tsv".format(folders[Indx]), recursive=True) # change [parent_directory] to the parent directory where the data is stored

# sort the list of files
clin_list.sort()
matching_columns = ['submitter_id.samples', 'age_at_index.demographic', 'age_at_diagnosis.diagnoses', 'gender.demographic', 'race.demographic', 'tumor_stage.diagnoses']
# Load the file
if len(clin_list) > 0:
    for file in clin_list:
        print("Loading file: " + file)
        df = pd.read_table(file)
        df = df[matching_columns]
else:
    print("No file matching the pattern found.")
print(len(clin_list))
# load the first file and print the shape
clin1 = clin_list[0]
clin = pd.read_table(clin1)
clin = clin[matching_columns]
print(clin.shape)
clin.head()

### 2. Remove NANs

In [ ]:
# Check if columns have the NANs
print(clin.isnull().sum())
print(clin['age_at_index.demographic'].isnull().sum())
print(clin['age_at_diagnosis.diagnoses'].isnull().sum())
print(clin['gender.demographic'].isnull().sum())
print(clin['race.demographic'].isnull().sum())
print(clin['tumor_stage.diagnoses'].isnull().sum())

In [ ]:
print(clin.shape)
# drop the rows with NANs in clin['age_at_index.demographic']
clin = clin.dropna(subset=['age_at_index.demographic'])
print(clin.shape)

In [ ]:
# Check if columns have the NANs
print(clin.isnull().sum())
print(clin['age_at_index.demographic'].isnull().sum())
print(clin['age_at_diagnosis.diagnoses'].isnull().sum())
print(clin['gender.demographic'].isnull().sum())
print(clin['race.demographic'].isnull().sum())
print(clin['tumor_stage.diagnoses'].isnull().sum())

### 3. Convert categorical variables to Numerical

#### Change Age columns to float

In [ ]:
# convert the 'age_at_index.demographic', 'age_at_diagnosis.diagnoses'column to float
clin['age_at_index.demographic'] = clin['age_at_index.demographic'].astype(float)
clin['age_at_diagnosis.diagnoses'] = clin['age_at_diagnosis.diagnoses'].astype(float)
# check the variable format with pandas dtypes.
print(clin.dtypes)
# find the number of samples having age>50
age50 = clin['age_at_index.demographic'] > 50
print(age50.value_counts())

#### Gender: male=1, female=2

In [ ]:
# check the number of unique values in 'gender.demographic' column
num_unique_genders = clin['gender.demographic'].nunique()
unique_genders = clin['gender.demographic'].unique()
# num_unique_genders = clin['gender.demographic'].nunique(dropna=False)
print(f"The 'gender.demographic' column has {num_unique_genders} unique values.")
print(f"The unique values are: {unique_genders}")
# count the number of each unique value
value_counts = clin['gender.demographic'].value_counts()
print(value_counts)
# convert the 'gender.demographic' to numerical values male=1, female=2
gender_mapping = {'male': 1, 'female': 2}
clin['gender'] = clin['gender.demographic'].map(gender_mapping)

#### Race: white=1, asian=2, black or african american=3, not reported=4, 'american indian or alaska native'=5, 'native hawaiian or other pacific islander'=6

In [ ]:
# check the number of unique values in clin['race.demographic' column
num_unique_race = clin['race.demographic'].nunique()
unique_race = clin['race.demographic'].unique()
print(f"The 'race.demographic' column has {num_unique_race} unique values.")
print(f"The unique values in the 'race.demographic' column are: {unique_race}")
# count the number of each unique value
value_counts = clin['race.demographic'].value_counts()
print(value_counts)
# convert the 'race.demographic' to numerical values white=1, asian=2, black or african american=3, not reported=4
race_mapping = {'white': 1, 'asian': 2, 'black or african american': 3, 'not reported': 4, 'american indian or alaska native':5}
clin['race'] = clin['race.demographic'].map(race_mapping)

#### Cancer stage: stage 0=1, is=10, stage i=10, stage ia=11, stage ib=12, i/ii nos=20, stage ii=20, stage iia=21, stage iib=22, stage iic=23, stage iii=30, stage iiia=31, stage iiib=32, stage iiic=33, stage iv=40, stage iva=41, stage ivb=42, stage ivc=43, not reported=50, stage x=50

In [ ]:
# check the number of unique values in 'tumor_stage.diagnoses' column
num_unique_tumors = clin['tumor_stage.diagnoses'].nunique()
unique_tumors = clin['tumor_stage.diagnoses'].unique()
print(f"The 'tumor_stage.diagnoses' column has {num_unique_tumors} unique values.")
print(f"The unique values are: {unique_tumors}")
# count the number of each unique value
value_counts = clin['tumor_stage.diagnoses'].value_counts()
print(value_counts)

In [ ]:
# convert the 'tumor_stage.diagnoses' to numerical values stage i=10, stage ii=20, stage iii=30, stage iv=40, not reported=50
tumor_mapping = {'stage 0': 1, 'is': 10, 'stage i': 10, 'stage ia': 11, 'stage ib': 12, 'stage ic': 13, 'stage ii': 20, 'i/ii nos': 20, 'stage iia': 21, 'stage iib': 22, 'stage iic': 23, 'stage iii': 30, 'stage iiia': 31, 'stage iiib': 32, 'stage iiic': 33, 'stage iv': 40, 'stage iva': 41, 'stage ivb': 42, 'stage ivc': 43, 'not reported': 50, 'stage x': 50}
clin['tumor_stage'] = clin['tumor_stage.diagnoses'].map(tumor_mapping)

In [ ]:
# drop the columns that are not needed
clin.drop(['age_at_diagnosis.diagnoses', 'gender.demographic', 'race.demographic', 'tumor_stage.diagnoses'], axis=1, inplace=True)
# rename the column age_at_diagnosis.demographic to age
clin.rename(columns={'age_at_index.demographic': 'age'}, inplace=True)
clin.rename(columns={'submitter_id.samples': 'sample'}, inplace=True)
clin.head()

In [ ]:
print(clin.shape)
# save the data to a csv file
clin.to_csv(f"{folders[Indx]}_clinical_data.csv", index=False)
print(f"Data saved to {folders[Indx]}_clinical_data.csv")

In [ ]:
# load the data from the csv file and confirm the shape
clinical = pd.read_csv(f"/home/80024223/data/Xena-GDC/preprocessed-data/{folders[Indx]}_clinical_data.csv")
print(clinical.shape)

## Combine the data modalities

### miRNA + Survival

In [ ]:
# combine the stemmirna and survival dataframes using the 'sample' column as the key and 'OS' and 'OS.time' as the columns
stemmirna_surv = pd.merge(stemmirna, surv, on='sample', how='outer')
# remove the '_PATIENT' column
stemmirna_surv = stemmirna_surv.drop(columns='_PATIENT')
# Move the OS, OS.time to second and third columns of the dataframe
cols = list(stemmirna_surv.columns)
cols = [cols[0]] + [cols[-1]] + [cols[-2]] + cols[1:-2]
stemmirna_surv = stemmirna_surv[cols]
# replace the NaN values with 0
stemmirna_surv = stemmirna_surv.fillna(0)
stemmirna_surv

### DNA Methylation + miRNA + Survival

In [ ]:
# combine the hvdnameth and stemmirna_surv dataframes using the 'sample' column as the key but keep samples from both dataframes
# merge the dataframes
# dnameth_stemmirna_surv = pd.merge(stemmirna_surv, hvdnameth, on='sample', how='right')
dnameth_stemmirna_surv = pd.merge(stemmirna_surv, hvdnameth, on='sample', how='outer')
# replace the NaN values with 0
dnameth_stemmirna_surv = dnameth_stemmirna_surv.fillna(0)
dnameth_stemmirna_surv

### Gene Expression + DNA Methylation + miRNA + Survival

In [ ]:
# combine the genedataframe and dnameth_stemmirna_surv dataframes using the 'sample' column but keep samples from both dataframes
# merge the dataframes
gene_dnameth_stemmirna_surv = pd.merge(dnameth_stemmirna_surv, genedataframe, on='sample', how='outer')
# replace the NaN values with 0
gene_dnameth_stemmirna_surv = gene_dnameth_stemmirna_surv.fillna(0)
gene_dnameth_stemmirna_surv

## Save Combined Data to CSV

In [ ]:
# save the dataframe to a csv file without the index column
gene_dnameth_stemmirna_surv.to_csv(f'{folders[Indx]}_preprocessed_combined.csv', index=False)
print('File saved as: {}_preprocessed_combined.csv'.format(folders[Indx]))

### Load the combined 3 Modal (miRNA, DNA Methyl, Gene Expr) data and add the clinical data

In [ ]:
# create a list of folder names that will be used to look for the files
folders = ['TCGA-ACC', 'TCGA-BLCA', 'TCGA-BRCA', 'TCGA-CESC', 'TCGA-CHOL', 'TCGA-COAD', 'TCGA-DLBC',
 'TCGA-ESCA', 'TCGA-GBM', 'TCGA-HNSC', 'TCGA-KICH', 'TCGA-KIRC', 'TCGA-KIRP', 'TCGA-LAML', 'TCGA-LGG',
  'TCGA-LIHC', 'TCGA-LUAD', 'TCGA-LUSC', 'TCGA-MESO', 'TCGA-OV', 'TCGA-PAAD', 'TCGA-PCPG', 'TCGA-PRAD',
   'TCGA-READ', 'TCGA-SARC', 'TCGA-SKCM', 'TCGA-STAD', 'TCGA-TGCT', 'TCGA-THCA', 'TCGA-THYM', 'TCGA-UCEC',
    'TCGA-UCS', 'TCGA-UVM']

In [ ]:
# Index values that will be used for each cancer type
# change this to process a specific cancer type (0-32)
Indx = 0

In [ ]:
# load the csv and verify the dimensions of the saved file
preprocessed_data = pd.read_csv('{}_preprocessed_combined.csv'.format(folders[Indx]))
print('Opening file: {}_preprocessed_combined.csv'.format(folders[Indx]))
print(preprocessed_data.shape)

In [ ]:
# load the data from the csv file and confirm the shape
clinical = pd.read_csv(f"{folders[Indx]}_clinical_data.csv")
print("Loading file: {}_clinical_data.csv".format(folders[Indx]))
print(clinical.shape)
clinical.head()

In [ ]:
# combine the preprocessed_data and clinical dataframes using the 'sample' column but keep samples from both dataframes
combined_data_outer = pd.merge(preprocessed_data, clinical, on='sample', how='outer')
# replace the NaN values with 0
combined_data_outer = combined_data_outer.fillna(0)
print(combined_data_outer.shape)

In [ ]:
# combine the preprocessed_data and clinical dataframes using the 'sample' column but keep samples from both dataframes
combined_data_inner = pd.merge(preprocessed_data, clinical, on='sample', how='inner')
# replace the NaN values with 0
combined_data_inner = combined_data_inner.fillna(0)
print(combined_data_inner.shape)

In [ ]:
# find the samples different in the two dataframes
diff_samples = combined_data_outer[~combined_data_outer['sample'].isin(combined_data_inner['sample'])]
diff_samples

In [ ]:
# move the columns age	gender	race	tumor_stage to fourth, fifth, sixth and seventh columns in combined_data_inner
cols = list(combined_data_inner.columns)
cols = cols[:3] + cols[-4:] + cols[3:-4]
combined_data_inner = combined_data_inner[cols]
combined_data_inner

In [ ]:
# save the combined_data_inner dataframe to a csv file without the index column
combined_data_inner.to_csv(f'{folders[Indx]}_preprocessed_4modald_mR_Gen_DMeth_Clin.csv', index=False)
print('File saved as: {}_preprocessed_4modald_mR_Gen_DMeth_Clin.csv'.format(folders[Indx]))

## Divide the data into training and testing sets and save to CSV

In [ ]:
# create a list of folder names that will be used to look for the files
folders = ['TCGA-ACC', 'TCGA-BLCA', 'TCGA-BRCA', 'TCGA-CESC', 'TCGA-CHOL', 'TCGA-COAD', 'TCGA-DLBC',
 'TCGA-ESCA', 'TCGA-GBM', 'TCGA-HNSC', 'TCGA-KICH', 'TCGA-KIRC', 'TCGA-KIRP', 'TCGA-LAML', 'TCGA-LGG',
  'TCGA-LIHC', 'TCGA-LUAD', 'TCGA-LUSC', 'TCGA-MESO', 'TCGA-OV', 'TCGA-PAAD', 'TCGA-PCPG', 'TCGA-PRAD',
   'TCGA-READ', 'TCGA-SARC', 'TCGA-SKCM', 'TCGA-STAD', 'TCGA-TGCT', 'TCGA-THCA', 'TCGA-THYM', 'TCGA-UCEC',
    'TCGA-UCS', 'TCGA-UVM']

In [ ]:
# Index values that will be used for each cancer type
# change this to process a specific cancer type (0-32)
Indx = 0

In [ ]:
# load the data from the csv file and confirm the shape
preprocessed_4modald_mR_Gen_DMeth_Clin = pd.read_csv(f'{folders[Indx]}_preprocessed_4modald_mR_Gen_DMeth_Clin.csv')
print("Loading file: {}_preprocessed_4modald_mR_Gen_DMeth_Clin.csv".format(folders[Indx]))
print(preprocessed_4modald_mR_Gen_DMeth_Clin.shape)

In [ ]:
# Randomly divide the data into training (80%) and testing (20%0) sets and save to separate CSVs
# split the data into training and testing sets
train, test = train_test_split(preprocessed_4modald_mR_Gen_DMeth_Clin, test_size=0.2)
print(train.shape, test.shape)
# save the training and testing sets to csv files
train.to_csv(f'{folders[Indx]}_preprocessed_train_4modald_mR_Gen_DMeth_Clin.csv', index=False)
test.to_csv(f'{folders[Indx]}_preprocessed_test_4modald_mR_Gen_DMeth_Clin.csv', index=False)
print('Files saved as: {}_preprocessed_train_4modald_mR_Gen_DMeth_Clin.csv and {}_preprocessed_test_4modald_mR_Gen_DMeth_Clin.csv'.format(folders[Indx], folders[Indx]))

### ----------- End of Individual Cancer Data Preprocessing -----------

## Combine the Training data

In [ ]:
# create a list of folder names that will be used to look for the files
folders = ['TCGA-ACC', 'TCGA-BLCA', 'TCGA-BRCA', 'TCGA-CESC', 'TCGA-CHOL', 'TCGA-COAD', 'TCGA-DLBC',
 'TCGA-ESCA', 'TCGA-GBM', 'TCGA-HNSC', 'TCGA-KICH', 'TCGA-KIRC', 'TCGA-KIRP', 'TCGA-LAML', 'TCGA-LGG',
  'TCGA-LIHC', 'TCGA-LUAD', 'TCGA-LUSC', 'TCGA-MESO', 'TCGA-OV', 'TCGA-PAAD', 'TCGA-PCPG', 'TCGA-PRAD',
   'TCGA-READ', 'TCGA-SARC', 'TCGA-SKCM', 'TCGA-STAD', 'TCGA-TGCT', 'TCGA-THCA', 'TCGA-THYM', 'TCGA-UCEC',
    'TCGA-UCS', 'TCGA-UVM']

In [ ]:
# concatenate all the preprocessed training data files in folders list
# initialize empty dataframe
train = pd.DataFrame()
# concatenate all the dataframes in the list
for i in range(0, len(folders)):
    print(i, folders[i])
    train_file = pd.read_csv(f'{folders[i]}_preprocessed_train_4modald_mR_Gen_DMeth_Clin.csv')
    train = pd.concat([train, train_file])
    train = train.reset_index(drop=True)
    print('train shape:', train.shape)

In [ ]:
print('train shape:', train.shape)
train.head()
# remove the NaN values
train = train.fillna(0)
print('train shape:', train.shape)
train.head()

In [ ]:
# get the rows having OS.time=0
print(train[train['OS.time'] == 0])
# remove the rows having OS.time=0
Data_with_OS_time_0 = train[train['OS.time'] == 0]
OS_time_0 = train[train['OS.time'] == 0].index
train = train.drop(OS_time_0)
print('train shape:', train.shape, ', Data_with_OS_time_0 shape:', Data_with_OS_time_0.shape)

In [ ]:
# find the rows having all zero values
print(train[(train.iloc[:, 3:] == 0).all(axis=1)])
# remove the rows having all zero values
train = train[~(train.iloc[:, 3:] == 0).all(axis=1)]
print(train.shape)

In [ ]:
# save the train data to a csv file
train.to_csv('Combined_Train_Data_4modald_mR_Gen_DMeth_Clin.csv', index=False)
print('File saved as: Combined_Train_Data_4modald_mR_Gen_DMeth_Clin.csv')

##  Protein Expression data preprocessing

### 1. Load Files

In [ ]:
train_data = train
train_data.shape

In [ ]:
# load the train data
train_data = pd.read_csv('Combined_Train_Data_4modald_mR_Gen_DMeth_Clin.csv')
print('Opening file: Combined_Train_Data_4modald_mR_Gen_DMeth_Clin.csv')

In [ ]:
train_data.head()

### 2. Sanity Check: see if there are all zeros in a column and remove the constant features

In [ ]:
print(train_data.shape)
# check if there are any columns that have all zero values except the first three columns
print(train_data.columns[(train_data == 0).all()])
print(train_data.columns[3:][(train_data.iloc[:, 3:] == 0).all()])

In [ ]:
# view all rows of the printed columns
with pd.option_context('display.max_rows', None):
    print(train_data[['sample', 'hsa-mir-4297', 'hsa-mir-1302-8', 'hsa-mir-4293', 'hsa-mir-1184-3', 'hsa-mir-646', 'hsa-let-7a-3', 'hsa-mir-4252', 'hsa-mir-548i-2', 'hsa-mir-4330', 'hsa-mir-548a-1', 'hsa-mir-5787']])

In [ ]:
# remove the constant features
from feature_engine.selection import DropConstantFeatures
sel1 = DropConstantFeatures(tol=1, variables=None, missing_values='raise')
sel1.fit(train_data)
data_train = sel1.transform(train_data)
data_train.shape
data_train

In [ ]:
print(data_train.shape)
# check if there are any columns that have all zero values except the first three columns
print(data_train.columns[(data_train == 0).all()])
print(data_train.columns[3:][(data_train.iloc[:, 3:] == 0).all()])

In [ ]:
data_train.head()

### 3. Get the Sample names

In [ ]:
# get the sample names from train data
samples = data_train['sample']
print(samples.shape)
# sort the samples
samples = samples.sort_values()
pd.set_option('display.max_rows', None)
# reset the index
samples = samples.reset_index(drop=True)
print(samples)
print(len(samples))

### 4. The Protein Expression data for all cancer types is in a folder with patient_IDs as subdirectories

In [ ]:
# set the path of data folder
path = 'path_to_raw_data'
# list the number of folders in the data folder
import os
folders = os.listdir(path)
folders.sort()
print("List of Protein Expre data subfolders:", folders)
print("Total Patients:", len(folders))

### 5. Match the Sample names from other features with the patient_IDs having Protein Expression data

In [ ]:
# read the elements of samples and search through all subfolders of the path for a file starting with the sample name as samples*.tsv
# initialize empty dataframe
data = pd.DataFrame()
# iterate through the samples
for i in range(0, len(samples)):
    # iterate through the folders
    for j in range(0, len(folders)):
        # if string of sample except last 4 characters is the same as the folder name
        if samples[i][:-4] == folders[j]:
            print(j+1, 'Looking {} in folder {}'.format(samples[i], folders[j]))
            # get the list of files in the folder that has this structure: '{path}/{folders[j]}/'Protein Expression Quantification'/*'
            file = glob.glob(f'{path}/{folders[j]}/Protein Expression Quantification/*/{samples[i]}*')
            # read the file.tsv if it exists
            if len(file) > 0:
                print(file)
                # read the file columns having names 'AGID' and 'mutation'
                df = pd.read_csv(file[0], sep='\t', usecols=['AGID', 'protein_expression'])
                print('df shape: ', df.shape)
                # transpose the dataframe set AGID column as the column names and protein_expression as the values
                df = df.transpose()
                df = df.reset_index()
                # set the columns to the first row
                df.columns = df.iloc[0]
                # remove the first row
                df = df[1:]
                # rename the first column to 'sample'
                df = df.rename(columns={f'{df.columns[0]}':'sample'})
                # set the sample name as the sample[i]
                df['sample'] = samples[i]
                # concatenate the df to data dataframe
                data = pd.concat([data, df])
                # reset the index
                data = data.reset_index(drop=True)
                print('data shape: ', data.shape)
            else:
                print('File not found in folder:', folders[j])
#save the data to a csv file
data.to_csv('Protein_Expression_Train_Data.csv', index=False)

In [ ]:
print(data.shape)
data.head()

### 6. Impute the NANs with the mean of the column

In [ ]:
# for the second column onwards, impute the NANs with the mean of the column
data.iloc[:, 1:] = data.iloc[:, 1:].apply(lambda x: x.fillna(x.mean()), axis=0)
# check if there are any NaN values
print(data.isnull().sum().sum())
print(data.shape)
data.head()

In [ ]:
#save the data to a csv file
data.to_csv('Protein_Expression_Train_Data.csv', index=False)

### 7. Drop Constant Features (i.e. >99.8% similarity)

In [ ]:
protein_expr = data
print(protein_expr.shape)

In [ ]:
from feature_engine.selection import DropConstantFeatures
sel1 = DropConstantFeatures(tol=0.998, variables=None, missing_values='raise')
sel1.fit(protein_expr)
protein_expr = sel1.transform(protein_expr)
protein_expr.shape
protein_expr

### 8. Remove Colinear Features (i.e. >80% correlation)

In [ ]:
# temporary remove the first column for later adding it to the first column
protein_expr1 = protein_expr.iloc[:, 1:]
# check the variable format with pandas dtypes.
print(protein_expr1.dtypes)
# convert the variable to numerical variables
protein_expr1 = protein_expr1.astype(float)
# check the variable format with pandas dtypes.
print(protein_expr1.dtypes)

In [ ]:
# remove correlated features
from feature_engine.selection import SmartCorrelatedSelection
sel4 = SmartCorrelatedSelection(
    variables=None,
    method="pearson",
    threshold=0.8,
    missing_values="raise",
    selection_method="variance",
    estimator=None,
)

In [ ]:
print(protein_expr1.shape)
sel4.fit(protein_expr1)
protexpr = sel4.transform(protein_expr1)
protexpr.shape

### 9. Add OS labels and save the Protein Expression data as csv

In [ ]:
# add the 'sample' column from protein_expr to the first column of protexpr
protexpr.insert(0, 'sample', protein_expr['sample'])
# add OS and OS.time columns from data_train to the protexpr dataframe based on the sample name
protexpr = pd.merge(protexpr, data_train[['sample', 'OS', 'OS.time']], on='sample', how='inner')
# Move the OS, OS.time to second and third columns of the dataframe
cols = list(protexpr.columns)
cols = [cols[0]] + [cols[-1]] + [cols[-2]] + cols[1:-2]
protexpr = protexpr[cols]
print(protexpr.shape)
protexpr.head()

In [ ]:
#save the data to a csv file
protexpr.to_csv('Protein_Expression_Train_Data_processed.csv', index=False)

### 10. Visualize the data

In [ ]:
# Visualize the distribution of the OS.time variable
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="whitegrid")
plt.figure(figsize=(10, 6))
sns.histplot(data_train['OS.time'], kde=True)
plt.title('Distribution of OS.time')
plt.show()

In [ ]:
# Visualize the distribution of the first 4 variables
plt.figure(figsize=(10, 6))
sns.histplot(data_train[data_train.columns[3]], kde=True)
sns.histplot(data_train[data_train.columns[4]], kde=True)
sns.histplot(data_train[data_train.columns[5]], kde=True)
sns.histplot(data_train[data_train.columns[6]], kde=True)
plt.title('Distribution of ' + data_train.columns[3] + ', ' + data_train.columns[4] + ', ' + data_train.columns[5] + ', ' + data_train.columns[6])
plt.legend([data_train.columns[3], data_train.columns[4], data_train.columns[5], data_train.columns[6]])
plt.show()


### 11. Normalize the data

In [ ]:
# get the data_train.head() for 3rd columns onwards
data_train.iloc[:, 7:].head()

In [ ]:
# Normalize the data with zero mean and unit variance across the samples
from sklearn.preprocessing import StandardScaler
# initialize the standard scaler
scaler = StandardScaler()
# fit and transform the data without the first three columns
data_train.iloc[:, 7:] = scaler.fit_transform(data_train.iloc[:, 7:])
print('train shape:', data_train.shape)
data_train.head()

In [ ]:
# Visualize the distribution of the first 4 variables
plt.figure(figsize=(10, 6))
sns.histplot(data_train[data_train.columns[7]], kde=True)
sns.histplot(data_train[data_train.columns[8]], kde=True)
sns.histplot(data_train[data_train.columns[9]], kde=True)
sns.histplot(data_train[data_train.columns[10]], kde=True)
plt.title('Distribution of ' + data_train.columns[7] + ', ' + data_train.columns[8] + ', ' + data_train.columns[9] + ', ' + data_train.columns[10])
plt.legend([data_train.columns[7], data_train.columns[8], data_train.columns[9], data_train.columns[10]])
plt.show()


### 12. Add the Protein Expression data to the combined data

In [ ]:
# combine the protexpr and data_train dataframes using the 'sample', OS.time, and OS columns as the key but keep samples from both dataframes
print("Protein Expression Data shape: ", protexpr.shape)
print("Train Data shape: ", data_train.shape)
# merge the dataframes
protexpr_plus_train_data = pd.merge(protexpr, data_train, on=['sample', 'OS.time', 'OS'], how='outer')
print("Combined Train Data shape: ", protexpr_plus_train_data.shape)
# replace the NaN values with 0
protexpr_plus_train_data = protexpr_plus_train_data.fillna(0)
print(protexpr_plus_train_data.shape)
protexpr_plus_train_data.head()

In [ ]:
protexpr_plus_train_data.head()

### 13. Save the combined data to CSV

In [ ]:
# save to csv
protexpr_plus_train_data.to_csv('5Modal_Train_Data_mR_Gen_DMeth_Clin_Prot.csv', index=False)

In [ ]:
protexpr_plus_train_data = pd.read_csv('5Modal_Train_Data_mR_Gen_DMeth_Clin_Prot.csv')

In [ ]:
print(protexpr_plus_train_data.shape)
protexpr_plus_train_data.head()

In [ ]:
# get the columns having names 'age', 'gender', 'race', 'tumor_stage'
# Get the current column names as a list
cols = list(protexpr_plus_train_data.columns)
# Define the columns to move and their new positions
cols_to_move = ['age', 'gender', 'race', 'tumor_stage']
new_positions = [3, 4, 5, 6]
# Remove the columns to move from the current column list
for col in cols_to_move:
    cols.remove(col)
# Insert the columns to move at their new positions
for col, pos in zip(cols_to_move, new_positions):
    cols.insert(pos, col)
# Reindex the DataFrame with the new column order
protexpr_plus_train_data_tmp = protexpr_plus_train_data[cols]
protexpr_plus_train_data_tmp.head()

In [ ]:
print(protexpr_plus_train_data_tmp.shape)
fiveMod_Train_mR_Gen_DMeth_Clin_Prot = protexpr_plus_train_data_tmp
print(fiveMod_Train_mR_Gen_DMeth_Clin_Prot.shape)

### 14. Sanity Check: verify that there are no zero-valued rows or columns

In [ ]:
# check if there are any rows that have all Zero values
# find the rows having all zero values
print(fiveMod_Train_mR_Gen_DMeth_Clin_Prot[(fiveMod_Train_mR_Gen_DMeth_Clin_Prot.iloc[:, 7:] == 0).all(axis=1)])

In [ ]:
# check if there are any columns that have all Zero values
# find the columns having all zero values
print(fiveMod_Train_mR_Gen_DMeth_Clin_Prot.columns[(fiveMod_Train_mR_Gen_DMeth_Clin_Prot == 0).all()])

In [ ]:
# save the fiveMod_Train_mR_Gen_DMeth_Clin_Prot to csv
fiveMod_Train_mR_Gen_DMeth_Clin_Prot.to_csv('5Modal_Train_Data_mR_Gen_DMeth_Clin_Prot.csv', index=False)

### 15. k-Fold Cross Validation

In [ ]:
#get the patient names from sample column
patientnames=fiveMod_Train_mR_Gen_DMeth_Clin_Prot['sample']
print(len(patientnames))
print(patientnames)

In [ ]:
# Generate k-fold cross validation splits for the data, k=10
from sklearn.model_selection import KFold
# initialize the kfold object
kf = KFold(n_splits=10, shuffle=True, random_state=42)
folds = list(kf.split(patientnames))
folds_array = np.zeros((len(patientnames), 10))
for i in range(10):
    folds_array[folds[i][1], i] = 1
folds_df = pd.DataFrame(folds_array, columns=['fold_{}'.format(i) for i in range(1,11)])
folds_df.index = patientnames
#replace 0 with Train and 1 with Test
folds_df = folds_df.replace(0, 'Train')
folds_df = folds_df.replace(1, 'Test')
folds_df.to_csv('pnas_fiveMod_Train_mR_Gen_DMeth_Clin_Prot_splits.csv')
folds_df

### 15. Save the Train Data in a .pkl file for Training

In [ ]:
# save the samples, vital_status (OS) and labels (OS.time) to numpy.ndarray
samples = fiveMod_Train_mR_Gen_DMeth_Clin_Prot['sample'].values
vital_status = fiveMod_Train_mR_Gen_DMeth_Clin_Prot['OS'].values
survival = fiveMod_Train_mR_Gen_DMeth_Clin_Prot['OS.time'].values
print('samples:', samples, 'vital_status:', vital_status, 'labels:', survival)
print('samples shape:', samples.shape, 'vital_status shape:', vital_status.shape, 'survival shape:', survival.shape)
# convert survival to df
survival_df = pd.DataFrame(survival, columns=['Labels'])
survival_df.index = samples
survival_df = survival_df.astype(float)\
# convert vital_status to df
vital_status_df = pd.DataFrame(vital_status, columns=['Vital_status'])
vital_status_df.index = samples
vital_status_df = vital_status_df.astype(float)
# remove the vital_status (OS) and labels (OS.time) from the dataframe
print('train shape:', fiveMod_Train_mR_Gen_DMeth_Clin_Prot.shape)
omic = fiveMod_Train_mR_Gen_DMeth_Clin_Prot.drop(columns=['sample', 'OS', 'OS.time'])
omic.index = samples
print('omic shape:', omic.shape)
omic.head()

In [ ]:
data_cv = {}
data_cv['cv_splits'] = {}

for i in range(10):
    # add and store data for each fold in the data_cv dictionary
    data_cv['cv_splits'][i+1] = {}
    data_cv['cv_splits'][i+1]['train'] = {}
    data_cv['cv_splits'][i+1]['test'] = {}
    data_cv['cv_splits'][i+1]['train'] = {
        'x_patname': folds_df[folds_df['fold_{}'.format(i+1)] == 'Train'].index.values.tolist(),
        'x_omic': omic.loc[folds_df[folds_df['fold_{}'.format(i+1)] == 'Train'].index.values.tolist()].values,
        'e': np.squeeze(vital_status_df.loc[folds_df[folds_df['fold_{}'.format(i+1)] == 'Train'].index.values.tolist()].values),
        't': np.squeeze(survival_df.loc[folds_df[folds_df['fold_{}'.format(i+1)] == 'Train'].index.values.tolist()].values)
    }
    data_cv['cv_splits'][i+1]['test'] = {
        'x_patname': folds_df[folds_df['fold_{}'.format(i+1)] == 'Test'].index.values.tolist(),
        'x_omic': omic.loc[folds_df[folds_df['fold_{}'.format(i+1)] == 'Test'].index.values.tolist()].values,
        'e': np.squeeze(vital_status_df.loc[folds_df[folds_df['fold_{}'.format(i+1)] == 'Test'].index.values.tolist()].values),
        't': np.squeeze(survival_df.loc[folds_df[folds_df['fold_{}'.format(i+1)] == 'Test'].index.values.tolist()].values)
    }

# save the dictionary to a .pkl file
with open('omic_10cv.pkl', 'wb') as f:
    pickle.dump(data_cv, f)

In [ ]:
# VERIFY THE PKL FILE
import pickle
data_cv = pickle.load(open('omic_10cv.pkl', 'rb'))
data_cv_splits = data_cv['cv_splits']
for k, data in data_cv_splits.items():
	print("*******************************************")
	print("************** SPLIT (%d/%d) **************" % (k, len(data_cv_splits.items())))
	print("*******************************************")
	if k == 1:
		print(data_cv_splits)
	print(len(data_cv_splits[k]['train']['x_patname']), (data_cv_splits[k]['train']['e']).shape, (data_cv_splits[k]['train']['t']).shape, (data_cv_splits[k]['train']['x_omic']).shape)

In [ ]:
print('data_cv.keys: ', data_cv.keys())
print("data_cv['cv_splits'].keys: ", data_cv['cv_splits'].keys())
print("data_cv['cv_splits'][1]: ", data_cv['cv_splits'][1].keys())
print("data_cv['cv_splits'][1]['train']: ", data_cv['cv_splits'][1]['train'].keys())

### Since the single file is ~45GB, we will divide the data for each fold and save to separate .pkl files

In [ ]:
data = data_cv

In [ ]:
for i in range(1, 11):
    # create a new dictionary for each fold
    data_dict = {
        'cv_splits': {
            i: {
                'train': data['cv_splits'][i]['train'],
                'test': data['cv_splits'][i]['test']
            }
        }
    }
    with open('omic_10cv_fold_{}.pkl'.format(i), 'wb') as f:
        pickle.dump(data_dict, f)

In [ ]:
# load .pkl file for each fold and verify the data lengths
for k in range(1, 11):
    data = pickle.load(open('omic_10cv_fold_{}.pkl'.format(k), 'rb'))
    print("*******************************************")
    print("************** SPLIT (%d/%d) **************" % (k, 10))
    print("*******************************************")
    print('data.keys: ', data.keys())
    print("data['cv_splits'].keys: ", data['cv_splits'].keys())
    print("data['cv_splits'][1]: ", data['cv_splits'][k].keys())
    print("data['cv_splits'][1]['train']: ", data['cv_splits'][k]['train'].keys())
    print("data['cv_splits'][1]['test']: ", data['cv_splits'][k]['test'].keys())
    print(len(data['cv_splits'][k]['train']['x_patname']),
            (data['cv_splits'][k]['train']['e']).shape,
            (data['cv_splits'][k]['train']['t']).shape,
            (data['cv_splits'][k]['train']['x_omic']).shape
    )
    print(len(data['cv_splits'][k]['test']['x_patname']), (data['cv_splits'][k]['test']['e']).shape, (data['cv_splits'][k]['test']['t']).shape, (data['cv_splits'][k]['test']['x_omic']).shape)

##  DNA Mutation data preprocessing

### 1. Load Files

In [ ]:
fiveMod_Train_mR_Gen_DMeth_Clin_Prot.head()

### 2. Get the Sample names

In [ ]:
# get the sample names from train data
samples = fiveMod_Train_mR_Gen_DMeth_Clin_Prot['sample']
print(samples.shape)
# sort the samples
samples = samples.sort_values()
pd.set_option('display.max_rows', None)
# reset the index
samples = samples.reset_index(drop=True)
print(samples)
print(len(samples))

### 3. Load the DNA Mutation data

In [ ]:
# load DNA Mut data from DNA_union_df.csv
DNA_mut = pd.read_csv('DNA_union_df.csv')
print(DNA_mut.shape)

In [ ]:
DNA_mut.head()

### 4. Match the Sample names

In [ ]:
print(samples.shape)
print(DNA_mut['SampleID'].shape)
# get the DNA_mut data based on DNA_mut['SampleID']=samples[:-4]
DNA_mut_data = DNA_mut[DNA_mut['SampleID'].isin(samples.str[:-4])]
print(DNA_mut_data.shape)
DNA_mut_data.head()

### 5. Impute the NANs with the mean of the column

In [ ]:
# check if there are any NANs for second column onwards
print(DNA_mut_data.isnull().sum().sum())

### 6. Drop Constant Features (i.e. >100% similarity (all zeros))

In [ ]:
# check if any column has all zero values
print(DNA_mut_data.columns[(DNA_mut_data == 0).all()])
# get all the column names having all zero values in alist
all_zero_columns = DNA_mut_data.columns[(DNA_mut_data == 0).all()].tolist()
print(len(all_zero_columns))

In [ ]:
from feature_engine.selection import DropConstantFeatures
print(DNA_mut_data.shape)
sel1 = DropConstantFeatures(tol=1, variables=None, missing_values='raise')
sel1.fit(DNA_mut_data)
DNA_muts = sel1.transform(DNA_mut_data)
print(DNA_muts.shape)
DNA_muts.head()

### 7. Remove Colinear Features (i.e. >80% correlation)

In [ ]:
# temporary remove the first column for later adding it to the first column
DNA_muts1 = DNA_muts.iloc[:, 1:]
# check the variable format with pandas dtypes.
print(DNA_muts1.dtypes)
# convert the variable to numerical variables
DNA_muts1 = DNA_muts1.astype(float)
# check the variable format with pandas dtypes.
print(DNA_muts1.dtypes)

In [ ]:
# remove correlated features
from feature_engine.selection import SmartCorrelatedSelection
sel5 = SmartCorrelatedSelection(
    variables=None,
    method="pearson",
    threshold=0.8,
    missing_values="raise",
    selection_method="variance",
    estimator=None,
)

In [ ]:
print(DNA_muts1.shape)
sel5.fit(DNA_muts1)

In [ ]:
DNAmuts = sel5.transform(DNA_muts1)
DNAmuts.shape

In [ ]:
DNAmuts.head()

In [ ]:
# save the DNAmuts data to a csv file
DNAmuts.to_csv('DNA_Mut_Train_Data_processed.csv', index=False)

In [ ]:
# load file DNA_Mut_Train_Data_processed.csv
DNAmuts = pd.read_csv('DNA_Mut_Train_Data_processed.csv')

In [ ]:
print(DNAmuts.shape)
DNAmuts.head()

In [ ]:
actual_data = DNAmuts
print(actual_data.shape)
actual_data.head()

In [ ]:
temp = DNA_muts
temp.head()

In [ ]:
# reset the index of temp
temp = temp.reset_index(drop=True)
print(temp.shape)
temp.head()

In [ ]:
# add 'SampleID' from temp to the first column of actual_data
actual_data.insert(0, 'SampleID', temp['SampleID'])
print(actual_data.shape)
actual_data.head()

### 8. Add OS labels and save the DNA Mut data as csv

In [ ]:
# Add 'sample, 'OS', 'OS.time' from four_modal to DNAmuts
# get the fiveMod_Train_mR_Gen_DMeth_Clin_Prot['sample', 'OS', 'OS.time'] in a separate df
fivemodal_3 = fiveMod_Train_mR_Gen_DMeth_Clin_Prot[['sample', 'OS', 'OS.time']]
print(fivemodal_3.shape)
print(fivemodal_3.head())
# add fivemodal_3['sample'].str[:-4] to fivemodal_3 as first column
fivemodal_3.insert(0, 'SampleID', fivemodal_3['sample'].str[:-4])
print(fivemodal_3.shape)
print(fivemodal_3.head())

# add fiveMod_Train_mR_Gen_DMeth_Clin_Prot['sample] to actual_data as first column based on SampleID=sample[:-4]
DNAMut_data = pd.merge(actual_data, fivemodal_3[['SampleID', 'sample', 'OS', 'OS.time']], left_on='SampleID', right_on='SampleID', how='inner')
print(DNAMut_data.shape)
# Move the sample, OS, OS.time to first, second and third columns of the dataframe
cols = list(DNAMut_data.columns)
cols = [cols[-3]] + [cols[-1]] + [cols[-2]] + cols[0:-3]
DNAMut_data = DNAMut_data[cols]
print(DNAMut_data.shape)
DNAMut_data.head()

In [ ]:
# check the duplicate rows
print(DNAMut_data['SampleID'].duplicated().sum())
print(DNAMut_data['sample'].duplicated().sum())
# get the duplicate rows based on DNAMut_data['SampleID']
print(DNAMut_data[DNAMut_data['SampleID'].duplicated()])

In [ ]:
# remove the duplicate rows based on SampleID
print(DNAMut_data.shape)
DM_data = DNAMut_data.drop_duplicates(subset='sample', keep='first')
print(DM_data.shape)
print(DM_data['SampleID'].duplicated().sum())

In [ ]:
# remove the duplicate rows based on SampleID
print(DNAMut_data.shape)
DM_data = DNAMut_data.drop_duplicates(subset='SampleID', keep='first')
print(DM_data.shape)
print(DM_data['SampleID'].duplicated().sum())

In [ ]:
DM_data.head()

In [ ]:
# drop the SampleID column
DM_data = DM_data.drop(columns='SampleID')
print(DM_data.shape)
DM_data.head()

In [ ]:
# save to a csv
DM_data.to_csv('DNA_Mut_Train_Data_processed_final.csv', index=False)

In [ ]:
# load the csv and verify the dimensions of the saved file
# DM_data = pd.read_csv('DNA_Mut_Train_Data_processed_final.csv')
DM_data = pd.read_csv('DNA_Mut_Train_Data_processed_final.csv')
print('Opening file: DNA_Mut_Train_Data_processed_final.csv')
print(DM_data.shape)

In [ ]:
DM_data.head()

### 10. Normalize the data

##### We don't need to normalize, because the DNA Mut data is already binary, and the 4Modality data is already normalized

### 11. Add the DNA Mut data to the combined data

In [ ]:
# combine the DNA Mut and fiveMod_Train_mR_Gen_DMeth_Clin_Prot dataframes using the 'sample', OS.time, and OS columns as the key but keep samples from both dataframes
print("DNA Mut Data shape: ", DM_data.shape)
print("5Modal Train Data shape: ", fiveMod_Train_mR_Gen_DMeth_Clin_Prot.shape)
# merge the dataframes
sixmodal_train_data = pd.merge(DM_data, fiveMod_Train_mR_Gen_DMeth_Clin_Prot, on=['sample', 'OS.time', 'OS'], how='outer')
print("Combined Train Data shape: ", sixmodal_train_data.shape)
# replace the NaN values with 0
sixmodal_train_data = sixmodal_train_data.fillna(0)
print(sixmodal_train_data.shape)
sixmodal_train_data.head()

In [ ]:
# get the columns having names 'age', 'gender', 'race', 'tumor_stage'
# Get the current column names as a list
cols = list(sixmodal_train_data.columns)
# Define the columns to move and their new positions
cols_to_move = ['age', 'gender', 'race', 'tumor_stage']
new_positions = [3, 4, 5, 6]
# Remove the columns to move from the current column list
for col in cols_to_move:
    cols.remove(col)
# Insert the columns to move at their new positions
for col, pos in zip(cols_to_move, new_positions):
    cols.insert(pos, col)
# Reindex the DataFrame with the new column order
sixmodal_train_data_tmp = sixmodal_train_data[cols]
sixmodal_train_data_tmp.head()

### 12. Save the combined data to CSV

In [ ]:
# save to a csv
sixmodal_train_data_tmp.to_csv('6Modal_Train_Data_mR_Gen_DMeth_Clin_Prot_Mut.csv', index=False)
print("saving to 6Modal_Train_Data_mR_Gen_DMeth_Clin_Prot_Mut.csv")

In [ ]:
# load the csv file
sixmodal_train_data = pd.read_csv('6Modal_Train_Data_mR_Gen_DMeth_Clin_Prot_Mut.csv')
print('Opening file: 6Modal_Train_Data_mR_Gen_DMeth_Clin_Prot_Mut.csv')
print(sixmodal_train_data.shape)
sixmodal_train_data.head()

In [ ]:
print(sixmodal_train_data.shape)

### 13. Sanity Check: verify that there are no zero-valued rows or columns

In [ ]:
# check if there are any rows that have all Zero values
# find the rows having all zero values
print(sixmodal_train_data[(sixmodal_train_data.iloc[:, 3:] == 0).all(axis=1)])

In [ ]:
# check if there are any columns that have all Zero values
# find the columns having all zero values
print(sixmodal_train_data.columns[(sixmodal_train_data == 0).all()])

### 14. k-Fold Cross Validation

In [ ]:
#get the patient names from sample column
patientnames=sixmodal_train_data['sample']
print(len(patientnames))
print(patientnames)

In [ ]:
# Generate k-fold cross validation splits for the data, k=10
from sklearn.model_selection import KFold
# initialize the kfold object
kf = KFold(n_splits=10, shuffle=True, random_state=42)
folds = list(kf.split(patientnames))
folds_array = np.zeros((len(patientnames), 10))
for i in range(10):
    folds_array[folds[i][1], i] = 1
folds_df = pd.DataFrame(folds_array, columns=['fold_{}'.format(i) for i in range(1,11)])
folds_df.index = patientnames
#replace 0 with Train and 1 with Test
folds_df = folds_df.replace(0, 'Train')
folds_df = folds_df.replace(1, 'Test')
folds_df.to_csv('pnas_splits.csv')
folds_df

### 15. Save the Train Data in a .pkl file for Training

In [ ]:
# save the samples, vital_status (OS) and labels (OS.time) to numpy.ndarray
samples = sixmodal_train_data['sample'].values
vital_status = sixmodal_train_data['OS'].values
survival = sixmodal_train_data['OS.time'].values
print('samples:', samples, 'vital_status:', vital_status, 'labels:', survival)
print('samples shape:', samples.shape, 'vital_status shape:', vital_status.shape, 'survival shape:', survival.shape)
# convert survival to df
survival_df = pd.DataFrame(survival, columns=['Labels'])
survival_df.index = samples
survival_df = survival_df.astype(float)
# convert vital_status to df
vital_status_df = pd.DataFrame(vital_status, columns=['Vital_status'])
vital_status_df.index = samples
vital_status_df = vital_status_df.astype(float)
# remove the vital_status (OS) and labels (OS.time) from the dataframe
print('train shape:', sixmodal_train_data.shape)
omic = sixmodal_train_data.drop(columns=['sample', 'OS', 'OS.time'])
omic.index = samples
print('omic shape:', omic.shape)
omic.head()

In [ ]:
 # convert dataframes to numpy arrays
data_cv = {}
data_cv['cv_splits'] = {}

for i in range(10):
    # add and store data for each fold in the data_cv dictionary
    data_cv['cv_splits'][i+1] = {}
    data_cv['cv_splits'][i+1]['train'] = {}
    data_cv['cv_splits'][i+1]['test'] = {}
    data_cv['cv_splits'][i+1]['train'] = {
        'x_patname': folds_df[folds_df['fold_{}'.format(i+1)] == 'Train'].index.values.tolist(),
        'x_omic': omic.loc[folds_df[folds_df['fold_{}'.format(i+1)] == 'Train'].index.values.tolist()].values,
        'e': np.squeeze(vital_status_df.loc[folds_df[folds_df['fold_{}'.format(i+1)] == 'Train'].index.values.tolist()].values),
        't': np.squeeze(survival_df.loc[folds_df[folds_df['fold_{}'.format(i+1)] == 'Train'].index.values.tolist()].values)
    }
    data_cv['cv_splits'][i+1]['test'] = {
        'x_patname': folds_df[folds_df['fold_{}'.format(i+1)] == 'Test'].index.values.tolist(),
        'x_omic': omic.loc[folds_df[folds_df['fold_{}'.format(i+1)] == 'Test'].index.values.tolist()].values,
        'e': np.squeeze(vital_status_df.loc[folds_df[folds_df['fold_{}'.format(i+1)] == 'Test'].index.values.tolist()].values),
        't': np.squeeze(survival_df.loc[folds_df[folds_df['fold_{}'.format(i+1)] == 'Test'].index.values.tolist()].values)
    }

# save the dictionary to a .pkl file
with open('omic_10cv.pkl', 'wb') as f:
    pickle.dump(data_cv, f)

In [ ]:
# VERIFY THE PKL FILE
import pickle
data_cv = pickle.load(open('omic_10cv.pkl', 'rb'))
data_cv_splits = data_cv['cv_splits']

for k, data in data_cv_splits.items():
	print("*******************************************")
	print("************** SPLIT (%d/%d) **************" % (k, len(data_cv_splits.items())))
	print("*******************************************")
	if k == 1:
		print(data_cv_splits)
	print(len(data_cv_splits[k]['train']['x_patname']), (data_cv_splits[k]['train']['e']).shape, (data_cv_splits[k]['train']['t']).shape, (data_cv_splits[k]['train']['x_omic']).shape)

In [ ]:
# print the dictionary names, and sub-dictionary names
print('data_cv.keys: ', data_cv.keys())
print("data_cv['cv_splits'].keys: ", data_cv['cv_splits'].keys())
print("data_cv['cv_splits'][1]: ", data_cv['cv_splits'][1].keys())
print("data_cv['cv_splits'][1]['train']: ", data_cv['cv_splits'][1]['train'].keys())
print("data_cv['cv_splits'][1]['test']: ", data_cv['cv_splits'][1]['test'].keys())

### Since the single file is ~67GB, we will divide the data for each fold and save to separate .pkl files

In [ ]:
data = data_cv

In [ ]:
for i in range(1, 11):
    # create a new dictionary for each fold
    data_dict = {
        'cv_splits': {
            i: {
                'train': data['cv_splits'][i]['train'],
                'test': data['cv_splits'][i]['test']
            }
        }
    }

    # save the dictionary to a .pkl file
    with open('train/omic_10cv_fold_{}.pkl'.format(i), 'wb') as f:
        pickle.dump(data_dict, f)

In [ ]:
# load .pkl file for each fold and verify the data lengths
for k in range(1, 11):
    data = pickle.load(open('train/omic_10cv_fold_{}.pkl'.format(k), 'rb'))
    print("*******************************************")
    print("************** SPLIT (%d/%d) **************" % (k, 10))
    print("*******************************************")
    # print the dictionary names, and sub-dictionary names
    print('data.keys: ', data.keys())
    print("data['cv_splits'].keys: ", data['cv_splits'].keys())
    print("data['cv_splits'][1]: ", data['cv_splits'][k].keys())
    print("data['cv_splits'][1]['train']: ", data['cv_splits'][k]['train'].keys())
    print("data['cv_splits'][1]['test']: ", data['cv_splits'][k]['test'].keys())
    print(len(data['cv_splits'][k]['train']['x_patname']),
            (data['cv_splits'][k]['train']['e']).shape,
            (data['cv_splits'][k]['train']['t']).shape,
            (data['cv_splits'][k]['train']['x_omic']).shape
    )
    print(len(data['cv_splits'][k]['test']['x_patname']), (data['cv_splits'][k]['test']['e']).shape, (data['cv_splits'][k]['test']['t']).shape, (data['cv_splits'][k]['test']['x_omic']).shape)

## ----------- End of Train/Validation Data Preprocessing -----------

# Combine the Inference data

In [ ]:
import pandas as pd
import glob
import numpy as np
import pprint
import pickle
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from feature_engine.selection import DropDuplicateFeatures, DropConstantFeatures

In [ ]:
# create a list of folder names that will be used to look for the files
folders = ['TCGA-ACC', 'TCGA-BLCA', 'TCGA-BRCA', 'TCGA-CESC', 'TCGA-CHOL', 'TCGA-COAD', 'TCGA-DLBC',
 'TCGA-ESCA', 'TCGA-GBM', 'TCGA-HNSC', 'TCGA-KICH', 'TCGA-KIRC', 'TCGA-KIRP', 'TCGA-LAML', 'TCGA-LGG',
  'TCGA-LIHC', 'TCGA-LUAD', 'TCGA-LUSC', 'TCGA-MESO', 'TCGA-OV', 'TCGA-PAAD', 'TCGA-PCPG', 'TCGA-PRAD',
   'TCGA-READ', 'TCGA-SARC', 'TCGA-SKCM', 'TCGA-STAD', 'TCGA-TGCT', 'TCGA-THCA', 'TCGA-THYM', 'TCGA-UCEC',
    'TCGA-UCS', 'TCGA-UVM']

In [ ]:
# concatenate all the preprocessed test data files in folders list
# initialize empty dataframe
test = pd.DataFrame()
# concatenate all the dataframes in the list
for i in range(0, len(folders)):
    print(i, folders[i])
    test_file = pd.read_csv(f'{folders[i]}_preprocessed_test_4modald_mR_Gen_DMeth_Clin.csv')
    test = pd.concat([test, test_file])
    test = test.reset_index(drop=True)
    print('test shape:', test.shape)

In [ ]:
print('test shape:', test.shape)
test.head()
# remove the NaN values
test = test.fillna(0)
print('test shape:', test.shape)
test.head()

In [ ]:
# get the rows having OS.time=0
print(test[test['OS.time'] == 0])
# remove the rows having OS.time=0
Test_Data_with_OS_time_0 = test[test['OS.time'] == 0]
OS_time_0 = test[test['OS.time'] == 0].index
test = test.drop(OS_time_0)
print('test shape:', test.shape, ', Test_Data_with_OS_time_0 shape:', Test_Data_with_OS_time_0.shape)


In [ ]:
# find the rows having all zero values
print(test[(test.iloc[:, 3:] == 0).all(axis=1)])
# remove the rows having all zero values
test = test[~(test.iloc[:, 3:] == 0).all(axis=1)]
print(test.shape)

In [ ]:
# save the test data to a csv file
test.to_csv('Combined_Test_Data_4modald_mR_Gen_DMeth_Clin.csv', index=False)
print('File saved as: Combined_Test_Data_4modald_mR_Gen_DMeth_Clin.csv')

##  Protein Expression data preprocessing

### 1. Load Files

In [ ]:
test_data = test
test_data.shape

In [ ]:
# load the test data
test_data = pd.read_csv('Combined_Test_Data_4modald_mR_Gen_DMeth_Clin.csv')
print('Opening file: Combined_Test_Data_4modald_mR_Gen_DMeth_Clin.csv')

In [ ]:
test_data.head()

### 2. Sanity Check: see if there are all zeros in a column and remove the constant features

In [ ]:
print(test_data.shape)
test_data.head()
# check if there are any columns that have all zero values except the first three columns
print(test_data.columns[(test_data == 0).all()])
print(test_data.columns[3:][(test_data.iloc[:, 3:] == 0).all()])

In [ ]:
# there are features that have all zero values, but to keep the data consistent, we will only remove those features that have been removed from the training data
# Features removed from training data: 'hsa-mir-4297', 'hsa-mir-1302-8', 'hsa-mir-548f-5', 'hsa-mir-1184-3', 'hsa-let-7a-3', 'hsa-mir-4280', 'hsa-mir-548i-2', 'hsa-mir-4499', 'hsa-mir-4330', 'hsa-mir-3975', 'hsa-mir-5787'
# view all rows of the printed columns
with pd.option_context('display.max_rows', None):
    print(test_data[['sample', 'hsa-mir-4297', 'hsa-mir-1302-8', 'hsa-mir-4293', 'hsa-mir-1184-3', 'hsa-mir-646', 'hsa-let-7a-3', 'hsa-mir-4252', 'hsa-mir-548i-2', 'hsa-mir-4330', 'hsa-mir-548a-1', 'hsa-mir-5787']])
    # print the rows in the above columns that have non-zero values
    colms = ['sample', 'hsa-mir-4297', 'hsa-mir-1302-8', 'hsa-mir-4293', 'hsa-mir-1184-3', 'hsa-mir-646', 'hsa-let-7a-3', 'hsa-mir-4252', 'hsa-mir-548i-2', 'hsa-mir-4330', 'hsa-mir-548a-1', 'hsa-mir-5787']
    for column in colms[1:]:  # Skip the 'sample' column
        non_zero_samples = test_data.loc[test_data[column] != 0, 'sample']
        print(f'Samples in {column} with non-zero values:')
        print(non_zero_samples)

In [ ]:
print(test_data.shape)
test_data = test_data.drop(columns=['hsa-mir-4297', 'hsa-mir-1302-8', 'hsa-mir-4293', 'hsa-mir-1184-3', 'hsa-mir-646', 'hsa-let-7a-3', 'hsa-mir-4252', 'hsa-mir-548i-2', 'hsa-mir-4330', 'hsa-mir-548a-1', 'hsa-mir-5787'])
print(test_data.shape)

In [ ]:
test_data

In [ ]:
# save the csv
test_data.to_csv('TEMP_Combined_Test_Data_4modald_mR_Gen_DMeth_Clin.csv', index=False)
print('File saved as: TEMP_Combined_Test_Data_4modald_mR_Gen_DMeth_Clin.csv')

In [ ]:
# load the csv
test_data = pd.read_csv('TEMP_Combined_Test_Data_4modald_mR_Gen_DMeth_Clin.csv')
print(test_data.shape)

### 3. Get the Sample names

In [ ]:
# get the sample names from test data
samples = test_data['sample']
print(samples.shape)
# sort the samples
samples = samples.sort_values()
pd.set_option('display.max_rows', None)
# reset the index
samples = samples.reset_index(drop=True)
print(samples)
print(len(samples))

### 4. The Protein Expression data for all cancer types is in a folder with patient_IDs as subdirectories

In [ ]:
# set the path of data folder
path = 'TCGA-pancancer/data/raw'
# list the number of folders in the data folder
import os
folders = os.listdir(path)
folders.sort()
print("List of Protein Expre data subfolders:", folders)
print("Total Patients:", len(folders))

### 5. Match the Sample names from other features with the patient_IDs having Protein Expression data

In [ ]:
# read the elements of samples and search through all subfolders of the path for afile starting with the sample name as samples*.tsv
# initialize empty dataframe
data = pd.DataFrame()
# iterate through the samples
for i in range(0, len(samples)):
    # print(i+1, 'Sample: ', samples[i])
    # iterate through the folders
    for j in range(0, len(folders)):
        # if string of sample except last 4 characters is the same as the folder name
        if samples[i][:-4] == folders[j]:
            print(j+1, 'Looking {} in folder {}'.format(samples[i], folders[j]))
            # get the list of files in the folder that has this structure: '{path}/{folders[j]}/'Protein Expression Quantification'/*'
            file = glob.glob(f'{path}/{folders[j]}/Protein Expression Quantification/*/{samples[i]}*')
            # read the file.tsv if it exists
            if len(file) > 0:
                print(file)
                # read the file columns having names 'AGID' and 'mutation'
                df = pd.read_csv(file[0], sep='\t', usecols=['AGID', 'protein_expression'])
                print('df shape: ', df.shape)
                # transpose the dataframe set AGID column as the column names and protein_expression as the values
                df = df.transpose()
                df = df.reset_index()
                # set the columns to the first row
                df.columns = df.iloc[0]
                # remove the first row
                df = df[1:]
                # rename the first column to 'sample'
                df = df.rename(columns={f'{df.columns[0]}':'sample'})
                # set the sample name as the sample[i]
                df['sample'] = samples[i]
                # concatenate the df to data dataframe
                data = pd.concat([data, df])
                # reset the index
                data = data.reset_index(drop=True)
                print('data shape: ', data.shape)
            else:
                print('File not found in folder:', folders[j])
#save the data to a csv file
data.to_csv('Protein_Expression_Test_Data.csv', index=False)

In [ ]:
print(data.shape)
data.head()

### 6. Impute the NANs with the mean of the column

In [ ]:
# for the second column onwards, impute the NANs with the mean of the column
data.iloc[:, 1:] = data.iloc[:, 1:].apply(lambda x: x.fillna(x.mean()), axis=0)
# check if there are any NaN values
print(data.isnull().sum().sum())
print(data.shape)
data.head()

In [ ]:
#save the data to a csv file
data.to_csv('Protein_Expression_Test_Data.csv', index=False)

### 7. Drop Constant Features (i.e. >99.8% similarity)

In [ ]:
protein_expr_test = data
print(protein_expr_test.shape)

In [ ]:
from feature_engine.selection import DropConstantFeatures
sel1 = DropConstantFeatures(tol=0.998, variables=None, missing_values='raise')
sel1.fit(protein_expr_test)
protein_expr_test = sel1.transform(protein_expr_test)
protein_expr_test.shape

### 8. Remove Colinear Features (only those removed from the training data)

In [ ]:
# temporary remove the first column for later adding it to the first column
protein_expr_test1 = protein_expr_test.iloc[:, 1:]
# check the variable format with pandas dtypes.
print(protein_expr_test1.dtypes)
# convert the variable to numerical variables
protein_expr_test1 = protein_expr_test1.astype(float)
# check the variable format with pandas dtypes.
print(protein_expr_test1.dtypes)

In [ ]:
print(protein_expr_test1.shape)
print(protexpr.shape)
# Keep the columns in protein_expr_test1 that are in protexpr
protein_expr_test1 = protein_expr_test1[protexpr.columns]
print(protein_expr_test1.shape)

In [ ]:
# verify that the columns in protein_expr_test1 are the same as in protexpr
if protein_expr_test1.columns.equals(protexpr.columns):
    print("The columns in protein_expr_test1 and protexpr are the same.")
else:
    print("The columns in protein_expr_test1 and protexpr are not the same.")

### 9. Add OS labels and save the Protein Expression data as csv

In [ ]:
# add the 'sample' column from protein_expr_test to the first column of protein_expr_test1
protein_expr_test1.insert(0, 'sample', protein_expr_test['sample'])
# add OS and OS.time columns from test_data to the protein_expr_test1 dataframe based on the sample name
protein_expr_test1 = pd.merge(protein_expr_test1, test_data[['sample', 'OS', 'OS.time']], on='sample', how='inner')
# Move the OS, OS.time to second and third columns of the dataframe
cols = list(protein_expr_test1.columns)
cols = [cols[0]] + [cols[-1]] + [cols[-2]] + cols[1:-2]
protein_expr_test1 = protein_expr_test1[cols]
protein_expr_test = protein_expr_test1
print(protein_expr_test.shape)
protein_expr_test.head()

In [ ]:
protein_expr_test

In [ ]:
#save the data to a csv file
protein_expr_test.to_csv('Protein_Expression_Test_Data_processed.csv', index=False)

In [ ]:
# laod the csv
protein_expr_test = pd.read_csv('Protein_Expression_Test_Data_processed.csv')

### 10. Visualize the data

In [ ]:
# Visualize the distribution of the OS.time variable
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="whitegrid")
# plot distribution of OS.time
plt.figure(figsize=(10, 6))
sns.histplot(test_data['OS.time'], kde=True)
plt.title('Distribution of Test OS.time')
plt.show()

In [ ]:
# Visualize the distribution of the first 4 variables
plt.figure(figsize=(10, 6))
sns.histplot(test_data[test_data.columns[3]], kde=True)
sns.histplot(test_data[test_data.columns[4]], kde=True)
sns.histplot(test_data[test_data.columns[5]], kde=True)
sns.histplot(test_data[test_data.columns[6]], kde=True)
plt.title('Distribution of ' + test_data.columns[3] + ', ' + test_data.columns[4] + ', ' + test_data.columns[5] + ', ' + test_data.columns[6])
plt.legend([test_data.columns[3], test_data.columns[4], test_data.columns[5], test_data.columns[6]])
plt.xlabel('Expression')
plt.ylabel('Frequency')
plt.show()


### 11. Normalize the data

In [ ]:
# get the test_data.head() for 3rd columns onwards
test_data.iloc[:, 7:].head()

In [ ]:
# Normalize the data with zero mean and unit variance across the samples
from sklearn.preprocessing import StandardScaler
# initialize the standard scaler
scaler = StandardScaler()
# fit and transform the data without the first three columns
test_data.iloc[:, 7:] = scaler.fit_transform(test_data.iloc[:, 7:])
print('test shape:', test_data.shape)
test_data.head()

In [ ]:
# Visualize the distribution of the first 4 variables

plt.figure(figsize=(10, 6))
sns.histplot(test_data[test_data.columns[7]], kde=True)
sns.histplot(test_data[test_data.columns[8]], kde=True)
sns.histplot(test_data[test_data.columns[9]], kde=True)
sns.histplot(test_data[test_data.columns[10]], kde=True)
plt.title('Distribution of ' + test_data.columns[3] + ', ' + test_data.columns[4] + ', ' + test_data.columns[5] + ', ' + test_data.columns[6])
plt.legend([test_data.columns[7], test_data.columns[8], test_data.columns[9], test_data.columns[10]])
plt.xlabel('Expression')
plt.ylabel('Frequency')
plt.show()

### 12. Add the Protein Expression data to the combined data

In [ ]:
# combine the protein_expr_test and test_data dataframes using the 'sample', OS.time, and OS columns as the key but keep samples from both dataframes
print("Protein Expression Data shape: ", protein_expr_test.shape)
print("Train Data shape: ", test_data.shape)
# merge the dataframes
protexpr_plus_test_data = pd.merge(protein_expr_test, test_data, on=['sample', 'OS.time', 'OS'], how='outer')
print("Combined Train Data shape: ", protexpr_plus_test_data.shape)
# replace the NaN values with 0
protexpr_plus_test_data = protexpr_plus_test_data.fillna(0)
print(protexpr_plus_test_data.shape)
protexpr_plus_test_data.head()

### 13. Save the combined data to CSV

In [ ]:
# save to csv
protexpr_plus_test_data.to_csv('5Modal_Test_Data_mR_Gen_DMeth_Clin_Prot.csv', index=False)

In [ ]:
print(protexpr_plus_test_data.shape)
protexpr_plus_test_data.head()

In [ ]:
# get the columns having names 'age', 'gender', 'race', 'tumor_stage'
# Get the current column names as a list
cols = list(protexpr_plus_test_data.columns)
# Define the columns to move and their new positions
cols_to_move = ['age', 'gender', 'race', 'tumor_stage']
new_positions = [3, 4, 5, 6]
# Remove the columns to move from the current column list
for col in cols_to_move:
    cols.remove(col)
# Insert the columns to move at their new positions
for col, pos in zip(cols_to_move, new_positions):
    cols.insert(pos, col)
# Reindex the DataFrame with the new column order
protexpr_plus_test_data_tmp = protexpr_plus_test_data[cols]
protexpr_plus_test_data_tmp.head()

In [ ]:
print(protexpr_plus_test_data_tmp.shape)
fiveMod_Test_mR_Gen_DMeth_Clin_Prot = protexpr_plus_test_data_tmp
print(fiveMod_Test_mR_Gen_DMeth_Clin_Prot.shape)

### 14. Sanity Check: verify that there are no zero-valued rows or columns

In [ ]:
# check if there are any rows that have all Zero values
print(fiveMod_Test_mR_Gen_DMeth_Clin_Prot[(fiveMod_Test_mR_Gen_DMeth_Clin_Prot.iloc[:, 7:] == 0).all(axis=1)])

In [ ]:
# check if there are any columns that have all Zero values
print(fiveMod_Test_mR_Gen_DMeth_Clin_Prot.columns[(fiveMod_Test_mR_Gen_DMeth_Clin_Prot == 0).all()])

In [ ]:
# save the fiveMod_Test_mR_Gen_DMeth_Clin_Prot to csv
fiveMod_Test_mR_Gen_DMeth_Clin_Prot.to_csv('5Modal_Test_Data_mR_Gen_DMeth_Clin_Prot.csv', index=False)

In [ ]:
# load the csv
fiveMod_Test_mR_Gen_DMeth_Clin_Prot = pd.read_csv('5Modal_Test_Data_mR_Gen_DMeth_Clin_Prot.csv')

In [ ]:
print(fiveMod_Test_mR_Gen_DMeth_Clin_Prot.shape)
fiveMod_Test_mR_Gen_DMeth_Clin_Prot.head()

### 15. k-Fold Cross Validation

In [ ]:
#get the patient names from sample column
test_patientnames=fiveMod_Test_mR_Gen_DMeth_Clin_Prot['sample']
print(len(test_patientnames))
print(test_patientnames)

In [ ]:
# Generate k-fold cross validation splits for the data, k=10
from sklearn.model_selection import KFold
# initialize the kfold object
kf = KFold(n_splits=10, shuffle=True, random_state=42)
test_folds = list(kf.split(test_patientnames))
#save as .csv file having rows as patient names and columns as folds
test_folds_array = np.zeros((len(test_patientnames), 10))
for i in range(10):
    test_folds_array[test_folds[i][1], i] = 1
test_folds_df = pd.DataFrame(test_folds_array, columns=['fold_{}'.format(i) for i in range(1,11)])
test_folds_df.index = test_patientnames
#replace 0 with Train and 1 with Test
test_folds_df = test_folds_df.replace(0, 'Train')
test_folds_df = test_folds_df.replace(1, 'Test')
test_folds_df.to_csv('test/pnas_fiveMod_Test_mR_Gen_DMeth_Clin_Prot_splits.csv')
test_folds_df

### 16. Save the Test Data in a .pkl file for Inference

In [ ]:
# get the samples, vital_status (OS) and labels (OS.time) from the dataframe and assign to numpy.ndarray
# save the samples, vital_status (OS) and labels (OS.time) to numpy.ndarray
samples = fiveMod_Test_mR_Gen_DMeth_Clin_Prot['sample'].values
vital_status = fiveMod_Test_mR_Gen_DMeth_Clin_Prot['OS'].values
survival = fiveMod_Test_mR_Gen_DMeth_Clin_Prot['OS.time'].values
print('samples:', samples, 'vital_status:', vital_status, 'labels:', survival)
print('samples shape:', samples.shape, 'vital_status shape:', vital_status.shape, 'survival shape:', survival.shape)
# convert survival to df
survival_df = pd.DataFrame(survival, columns=['Labels'])
survival_df.index = samples
survival_df = survival_df.astype(float)
# convert vital_status to df
vital_status_df = pd.DataFrame(vital_status, columns=['Vital_status'])
vital_status_df.index = samples
vital_status_df = vital_status_df.astype(float)
# remove the vital_status (OS) and labels (OS.time) from the dataframe
print('train shape:', fiveMod_Test_mR_Gen_DMeth_Clin_Prot.shape)
omic = fiveMod_Test_mR_Gen_DMeth_Clin_Prot.drop(columns=['sample', 'OS', 'OS.time'])
omic.index = samples
print('omic shape:', omic.shape)
omic.head()

In [ ]:
data_cv = {}
data_cv['cv_splits'] = {}

for i in range(10):
    # add and store data for each fold in the data_cv dictionary
    data_cv['cv_splits'][i+1] = {}
    data_cv['cv_splits'][i+1]['train'] = {}
    data_cv['cv_splits'][i+1]['test'] = {}
    data_cv['cv_splits'][i+1]['train'] = {
        'x_patname': test_folds_df[test_folds_df['fold_{}'.format(i+1)] == 'Train'].index.values.tolist(),
        'x_omic': omic.loc[test_folds_df[test_folds_df['fold_{}'.format(i+1)] == 'Train'].index.values.tolist()].values,
        'e': np.squeeze(vital_status_df.loc[test_folds_df[test_folds_df['fold_{}'.format(i+1)] == 'Train'].index.values.tolist()].values),
        't': np.squeeze(survival_df.loc[test_folds_df[test_folds_df['fold_{}'.format(i+1)] == 'Train'].index.values.tolist()].values)
    }
    data_cv['cv_splits'][i+1]['test'] = {
        'x_patname': test_folds_df[test_folds_df['fold_{}'.format(i+1)] == 'Test'].index.values.tolist(),
        'x_omic': omic.loc[test_folds_df[test_folds_df['fold_{}'.format(i+1)] == 'Test'].index.values.tolist()].values,
        'e': np.squeeze(vital_status_df.loc[test_folds_df[test_folds_df['fold_{}'.format(i+1)] == 'Test'].index.values.tolist()].values),
        't': np.squeeze(survival_df.loc[test_folds_df[test_folds_df['fold_{}'.format(i+1)] == 'Test'].index.values.tolist()].values)
    }

# save the dictionary to a .pkl file
with open('test/omic_test_10cv_all.pkl', 'wb') as f:
    pickle.dump(data_cv, f)

In [ ]:
# VERIFY THE PKL FILE
#load the .pkl file
import pickle
data_cv = pickle.load(open('test/omic_test_10cv_all.pkl', 'rb'))
data_cv_splits = data_cv['cv_splits']

for k, data in data_cv_splits.items():
	print("*******************************************")
	print("************** SPLIT (%d/%d) **************" % (k, len(data_cv_splits.items())))
	print("*******************************************")
	if k == 1:
		print(data_cv_splits)
	print(len(data_cv_splits[k]['train']['x_patname']), (data_cv_splits[k]['train']['e']).shape, (data_cv_splits[k]['train']['t']).shape, (data_cv_splits[k]['train']['x_omic']).shape)

In [ ]:
print('data_cv.keys: ', data_cv.keys())
print("data_cv['cv_splits'].keys: ", data_cv['cv_splits'].keys())
print("data_cv['cv_splits'][1]: ", data_cv['cv_splits'][1].keys())
print("data_cv['cv_splits'][1]['test']: ", data_cv['cv_splits'][1]['test'].keys())

### Since the single file is ~14GB, we will divide the data for each fold and save to separate .pkl files

In [ ]:
data = data_cv

In [ ]:
for i in range(1, 11):
    # create a new dictionary for each fold
    data_dict = {
        'cv_splits': {
            i: {
                'train': data['cv_splits'][i]['train'],
                'test': data['cv_splits'][i]['test']
            }
        }
    }

    # save the dictionary to a .pkl file
    with open('test/omic_test_10cv_fold_{}.pkl'.format(i), 'wb') as f:
        pickle.dump(data_dict, f)

In [ ]:
# load .pkl file for each fold and verify the data lengths
for k in range(1, 11):
    data = pickle.load(open('test/omic_test_10cv_fold_{}.pkl'.format(k), 'rb'))
    print("*******************************************")
    print("************** SPLIT (%d/%d) **************" % (k, 10))
    print("*******************************************")
    print('data.keys: ', data.keys())
    print("data['cv_splits'].keys: ", data['cv_splits'].keys())
    print("data['cv_splits'][1]: ", data['cv_splits'][k].keys())
    print("data['cv_splits'][1]['train']: ", data['cv_splits'][k]['train'].keys())
    print("data['cv_splits'][1]['test']: ", data['cv_splits'][k]['test'].keys())
    print(len(data['cv_splits'][k]['train']['x_patname']),
            (data['cv_splits'][k]['train']['e']).shape,
            (data['cv_splits'][k]['train']['t']).shape,
            (data['cv_splits'][k]['train']['x_omic']).shape
    )
    print(len(data['cv_splits'][k]['test']['x_patname']), (data['cv_splits'][k]['test']['e']).shape, (data['cv_splits'][k]['test']['t']).shape, (data['cv_splits'][k]['test']['x_omic']).shape)

##  DNA Mutation data preprocessing

### 1. Load Files

In [ ]:
fiveMod_Test_mR_Gen_DMeth_Clin_Prot.head()

### 2. Get the Sample names

In [ ]:
# get the sample names from test data
samples = fiveMod_Test_mR_Gen_DMeth_Clin_Prot['sample']
print(samples.shape)
# sort the samples
samples = samples.sort_values()
pd.set_option('display.max_rows', None)
# reset the index
samples = samples.reset_index(drop=True)
print(samples)
print(len(samples))

### 3. Load the DNA Mutation data

In [ ]:
# load DNA Mut data from DNA_mut_data/DNA_union_df.csv
DNA_mut = pd.read_csv('DNA_mut_data/DNA_union_df.csv')
print('Opening file: DNA_mut_data/DNA_union_df.csv')
print(DNA_mut.shape)
DNA_mut.head()

### 4. Match the Sample names

In [ ]:
print(samples.shape)
print(DNA_mut['SampleID'].shape)
# get the DNA_mut data based on DNA_mut['SampleID']=samples[:-4]
DNA_mut_test = DNA_mut[DNA_mut['SampleID'].isin(samples.str[:-4])]
print(DNA_mut_test.shape)
DNA_mut_test.head()

### 5. Impute the NANs with the mean of the column

In [ ]:
# check if there are any NANs for second column onwards
print(DNA_mut_test.isnull().sum().sum())

### 6. Drop Constant Features (i.e. >100% similarity (all zeros))

In [ ]:
# check if any column has all zero values
print(DNA_mut_test.columns[(DNA_mut_test == 0).all()])
# get all the column names having all zero values in alist
all_zero_columns = DNA_mut_test.columns[(DNA_mut_test == 0).all()].tolist()
print(len(all_zero_columns))

#### Let's keep the same feature size as in training data

In [ ]:
# load the train data
# load file DNA_Mut_Train_Data_processed.csv
DNAmuts_train = pd.read_csv('DNA_Mut_Train_Data_processed.csv')
print('Opening file: DNA_Mut_Train_Data_processed.csv')
print(DNAmuts_train.shape)
DNAmuts_train.head()

#### Drop the features from test data that are not in training data

In [ ]:
print("DNA Mut Test data shape:", DNA_mut_test.shape)
print("DNA Mut Train data shape:", DNAmuts_train.shape)
# keep the SampleID column in the DNA_mut_test dataframe
SampleIDs = DNA_mut_test['SampleID']
# Drop the features from DNA_mut_test (except SampleID column) that are not in DNAmuts_train
DNA_muts_test = DNA_mut_test.drop(columns=[col for col in DNA_mut_test.columns if col not in DNAmuts_train.columns])
print("DNA Muts Test data shape:", DNA_muts_test.shape)
print(DNA_muts_test.head())
# add the SampleID column to the DNA_muts_test dataframe
DNA_muts_test.insert(0, 'SampleID', SampleIDs)
print("DNA Muts Test data shape:", DNA_muts_test.shape)
print(DNA_muts_test.head())
# reset the index
DNA_muts_test = DNA_muts_test.reset_index(drop=True)
DNA_muts_test.head()

In [ ]:
print(DNA_muts_test.shape)
DNA_muts_test.head()

### 8. Add OS labels and save the DNA Mut data as csv

In [ ]:
# Add 'sample, 'OS', 'OS.time' from four_modal_test to DNA_muts_test
# get the four_modal_test['sample', 'OS', 'OS.time'] in a separate df
fivemodal_test_3 = fiveMod_Test_mR_Gen_DMeth_Clin_Prot[['sample', 'OS', 'OS.time']]
print(fivemodal_test_3.shape)
print(fivemodal_test_3.head())
# add fivemodal_test_3['sample'].str[:-4] to fivemodal_test_3 as first column
fivemodal_test_3.insert(0, 'SampleID', fivemodal_test_3['sample'].str[:-4])
print(fivemodal_test_3.shape)
print(fivemodal_test_3.head())
# add fivemodal_test_3['sample] to DNA_muts_test as first column based on SampleID=sample[:-4]
DNAMut_test_data = pd.merge(DNA_muts_test, fivemodal_test_3[['SampleID', 'sample', 'OS', 'OS.time']], left_on='SampleID', right_on='SampleID', how='inner')
print(DNAMut_test_data.shape)
# Move the sample, OS, OS.time to first, second and third columns of the dataframe
cols = list(DNAMut_test_data.columns)
cols = [cols[-3]] + [cols[-1]] + [cols[-2]] + cols[0:-3]
DNAMut_test_data = DNAMut_test_data[cols]
print(DNAMut_test_data.shape)
DNAMut_test_data.head()

In [ ]:
# check the duplicate rows
print(DNAMut_test_data['SampleID'].duplicated().sum())
print(DNAMut_test_data['sample'].duplicated().sum())
# get the duplicate rows based on DNAMut_data['SampleID']
print(DNAMut_test_data[DNAMut_test_data['SampleID'].duplicated()])

In [ ]:
# remove the duplicate rows based on sample
print(DNAMut_test_data.shape)
DM_test_data = DNAMut_test_data.drop_duplicates(subset='sample', keep='first')
print(DM_test_data.shape)
print(DM_test_data['SampleID'].duplicated().sum())

In [ ]:
DM_test_data.head()

In [ ]:
# drop the SampleID column
DM_test_data = DM_test_data.drop(columns='SampleID')
print(DM_test_data.shape)
DM_test_data.head()

In [ ]:
# save to a csv
DM_test_data.to_csv('DNA_Mut_Test_Data_processed_final.csv', index=False)

In [ ]:
# load the csv and verify the dimensions of the saved file
DM_test_data = pd.read_csv('DNA_Mut_Test_Data_processed_final.csv')
print('Opening file: DNA_Mut_Test_Data_processed_final.csv')
print(DM_test_data.shape)
DM_test_data.head()

### 10. Normalize the data

##### We don't need to normalize, because the DNA Mut data is already binary, and the 4Modality data is already normalized

### 11. Add the DNA Mut data to the combined data (four_modal)

In [ ]:
# combine the DNA Mut and fiveMod_Test_mR_Gen_DMeth_Clin_Prot dataframes using the 'sample', OS.time, and OS columns as the key but keep samples from both dataframes
print("DNA Mut Data shape: ", DM_test_data.shape)
print("5Modal Test Data shape: ", fiveMod_Test_mR_Gen_DMeth_Clin_Prot.shape)
# merge the dataframes
sixmodal_test_data = pd.merge(DM_test_data, fiveMod_Test_mR_Gen_DMeth_Clin_Prot, on=['sample', 'OS.time', 'OS'], how='outer')
print("Combined Test Data shape: ", sixmodal_test_data.shape)
# replace the NaN values with 0
sixmodal_test_data = sixmodal_test_data.fillna(0)
print(sixmodal_test_data.shape)
sixmodal_test_data.head()

In [ ]:
# get the columns having names 'age', 'gender', 'race', 'tumor_stage'
# Get the current column names as a list
cols = list(sixmodal_test_data.columns)
# Define the columns to move and their new positions
cols_to_move = ['age', 'gender', 'race', 'tumor_stage']
new_positions = [3, 4, 5, 6]
# Remove the columns to move from the current column list
for col in cols_to_move:
    cols.remove(col)
# Insert the columns to move at their new positions
for col, pos in zip(cols_to_move, new_positions):
    cols.insert(pos, col)
# Reindex the DataFrame with the new column order
sixmodal_test_data_tmp = sixmodal_test_data[cols]
sixmodal_test_data_tmp.head()

### 12. Save the combined data to CSV

In [ ]:
# save to a csv
sixmodal_test_data_tmp.to_csv('6Modal_Test_Data_mR_Gen_DMeth_Clin_Prot_Mut.csv', index=False)
print('saving to 6Modal_Test_Data_mR_Gen_DMeth_Clin_Prot_Mut.csv')

### 13. Sanity Check: verify that there are no zero-valued rows or columns

In [ ]:
sixmodal_test_data = sixmodal_test_data_tmp

In [ ]:
# check if there are any rows that have all Zero values
# find the rows having all zero values
print(sixmodal_test_data[(sixmodal_test_data.iloc[:, 3:] == 0).all(axis=1)])

In [ ]:
# check if there are any columns that have all Zero values
# find the columns having all zero values
print(sixmodal_test_data.columns[(sixmodal_test_data == 0).all()])

### 14. k-Fold Cross Validation

In [ ]:
#get the patient names from sample column
test_patientnames=sixmodal_test_data['sample']
print(len(test_patientnames))
print(test_patientnames)

In [ ]:
# Generate k-fold cross validation splits for the data, k=10
from sklearn.model_selection import KFold
# initialize the kfold object
kf = KFold(n_splits=10, shuffle=True, random_state=42)
folds = list(kf.split(test_patientnames))
#save as .csv file having rows as patient names and columns as folds
folds_array = np.zeros((len(test_patientnames), 10))
for i in range(10):
    folds_array[folds[i][1], i] = 1
folds_df = pd.DataFrame(folds_array, columns=['fold_{}'.format(i) for i in range(1,11)])
folds_df.index = test_patientnames
#replace 0 with Train and 1 with Test
folds_df = folds_df.replace(0, 'Train')
folds_df = folds_df.replace(1, 'Test')
folds_df.to_csv('test/pnas_test_splits.csv')
folds_df

### 15. Save the Test Data in a .pkl file for Inference

In [ ]:
# save the samples, vital_status (OS) and labels (OS.time) to numpy.ndarray
samples = sixmodal_test_data['sample'].values
vital_status = sixmodal_test_data['OS'].values
survival = sixmodal_test_data['OS.time'].values
print('samples:', samples, 'vital_status:', vital_status, 'labels:', survival)
print('samples shape:', samples.shape, 'vital_status shape:', vital_status.shape, 'survival shape:', survival.shape)
# convert survival to df
survival_df = pd.DataFrame(survival, columns=['Labels'])
survival_df.index = samples
survival_df = survival_df.astype(float)
# convert vital_status to df
vital_status_df = pd.DataFrame(vital_status, columns=['Vital_status'])
vital_status_df.index = samples
vital_status_df = vital_status_df.astype(float)
# remove the vital_status (OS) and labels (OS.time) from the dataframe
print('test shape:', sixmodal_test_data.shape)
omic = sixmodal_test_data.drop(columns=['sample', 'OS', 'OS.time'])
omic.index = samples
print('omic shape:', omic.shape)
omic.head()

In [ ]:
 # convert dataframes to numpy arrays
data_cv = {}
data_cv['cv_splits'] = {}

for i in range(10):
    # add and store data for each fold in the data_cv dictionary
    data_cv['cv_splits'][i+1] = {}
    data_cv['cv_splits'][i+1]['train'] = {}
    data_cv['cv_splits'][i+1]['test'] = {}
    data_cv['cv_splits'][i+1]['train'] = {
        'x_patname': folds_df[folds_df['fold_{}'.format(i+1)] == 'Train'].index.values.tolist(),
        'x_omic': omic.loc[folds_df[folds_df['fold_{}'.format(i+1)] == 'Train'].index.values.tolist()].values,
        'e': np.squeeze(vital_status_df.loc[folds_df[folds_df['fold_{}'.format(i+1)] == 'Train'].index.values.tolist()].values),
        't': np.squeeze(survival_df.loc[folds_df[folds_df['fold_{}'.format(i+1)] == 'Train'].index.values.tolist()].values)
    }
    data_cv['cv_splits'][i+1]['test'] = {
        'x_patname': folds_df[folds_df['fold_{}'.format(i+1)] == 'Test'].index.values.tolist(),
        'x_omic': omic.loc[folds_df[folds_df['fold_{}'.format(i+1)] == 'Test'].index.values.tolist()].values,
        'e': np.squeeze(vital_status_df.loc[folds_df[folds_df['fold_{}'.format(i+1)] == 'Test'].index.values.tolist()].values),
        't': np.squeeze(survival_df.loc[folds_df[folds_df['fold_{}'.format(i+1)] == 'Test'].index.values.tolist()].values)
    }

# save the dictionary to a .pkl file
with open('test/omic_test_10cv.pkl', 'wb') as f:
    pickle.dump(data_cv, f)

In [ ]:
# VERIFY THE PKL FILE
#load the .pkl file
import pickle
data_cv = pickle.load(open('test/omic_test_10cv.pkl', 'rb'))
data_cv_splits = data_cv['cv_splits']

for k, data in data_cv_splits.items():
	print("*******************************************")
	print("************** SPLIT (%d/%d) **************" % (k, len(data_cv_splits.items())))
	print("*******************************************")
	if k == 1:
		print(data_cv_splits)
	print(len(data_cv_splits[k]['test']['x_patname']), (data_cv_splits[k]['test']['e']).shape, (data_cv_splits[k]['test']['t']).shape, (data_cv_splits[k]['test']['x_omic']).shape)

In [ ]:
print('data_cv.keys: ', data_cv.keys())
print("data_cv['cv_splits'].keys: ", data_cv['cv_splits'].keys())
print("data_cv['cv_splits'][1]: ", data_cv['cv_splits'][1].keys())
print("data_cv['cv_splits'][1]['train']: ", data_cv['cv_splits'][1]['train'].keys())
print("data_cv['cv_splits'][1]['test']: ", data_cv['cv_splits'][1]['test'].keys())

### Since the single file is ~17GB, we will divide the data for each fold and save to separate .pkl files

In [ ]:
data = data_cv

In [ ]:
for i in range(1, 11):
    # create a new dictionary for each fold
    data_dict = {
        'cv_splits': {
            i: {
                'train': data['cv_splits'][i]['train'],
                'test': data['cv_splits'][i]['test']
            }
        }
    }

    # save the dictionary to a .pkl file
    with open('test/omic_test_10cv_fold_{}.pkl'.format(i), 'wb') as f:
        pickle.dump(data_dict, f)

In [ ]:
# load .pkl file for each fold and verify the data lengths
for k in range(1, 11):
    data = pickle.load(open('test/omic_test_10cv_fold_{}.pkl'.format(k), 'rb'))
    print("*******************************************")
    print("************** SPLIT (%d/%d) **************" % (k, 10))
    print("*******************************************")
    print('data.keys: ', data.keys())
    print("data['cv_splits'].keys: ", data['cv_splits'].keys())
    print("data['cv_splits'][1]: ", data['cv_splits'][k].keys())
    print("data['cv_splits'][1]['train']: ", data['cv_splits'][k]['train'].keys())
    print("data['cv_splits'][1]['test']: ", data['cv_splits'][k]['test'].keys())
    print(len(data['cv_splits'][k]['train']['x_patname']),
            (data['cv_splits'][k]['train']['e']).shape,
            (data['cv_splits'][k]['train']['t']).shape,
            (data['cv_splits'][k]['train']['x_omic']).shape
    )
    print(len(data['cv_splits'][k]['test']['x_patname']), (data['cv_splits'][k]['test']['e']).shape, (data['cv_splits'][k]['test']['t']).shape, (data['cv_splits'][k]['test']['x_omic']).shape)

## Let's combine all folds into one pkl for the test samples only

In [ ]:
# load the .pkl file omic_test_10cv.pkl
import pickle
data = pickle.load(open('test/omic_test_10cv.pkl', 'rb'))
print('opening file: test/omic_test_10cv.pkl')
print('data.keys: ', data.keys())

In [ ]:
for k in range(1, 11):
    print(len(data['cv_splits'][k]['test']['x_patname']), (data['cv_splits'][k]['test']['e']).shape, (data['cv_splits'][k]['test']['t']).shape, (data['cv_splits'][k]['test']['x_omic']).shape)

In [ ]:
combined_test_data = {
            'cv_splits': {
                1: {
                    'test': {'x_patname': [], 'x_omic': [], 'e': [], 't': []} 
                }
            }
        }
print(combined_test_data)

import numpy as np

for key in data['cv_splits'].keys():
    print('key: ', key)
    for subkey in data['cv_splits'][key]['test'].keys():
        # print('subkey :', subkey)
        subdata = data['cv_splits'][key]['test'][subkey]
        if np.isscalar(subdata) or np.ndim(subdata) == 0:
            subdata = [subdata]
        combined_test_data['cv_splits'][1]['test'][subkey].extend(subdata)

# Save combined data into a new .pkl file
with open('test/omic_test_combined.pkl', 'wb') as f:
    print('saving as test/omic_test_combined.pkl')
    pickle.dump(combined_test_data, f)

In [ ]:
# Load the saved pkl file
data_test = pickle.load(open('test/omic_test_combined.pkl', 'rb'))
print('opening file: test/omic_test_combined.pkl')
print('data.keys: ', data_test.keys())
print("data_cv['cv_splits'].keys: ", data_test['cv_splits'].keys())
print("data_cv['cv_splits'][1]: ", data_test['cv_splits'][1].keys())
print("data_cv['cv_splits'][1]['test']: ", data_test['cv_splits'][1]['test'].keys())
print("data_cv['cv_splits'][1]['test']['x_patname']: ", len(data_test['cv_splits'][1]['test']['x_patname']))
print("data_cv['cv_splits'][1]['test']['x_omic']: ", len(data_test['cv_splits'][1]['test']['x_omic']))
print("data_cv['cv_splits'][1]['test']['e']: ", len(data_test['cv_splits'][1]['test']['e']))
print("data_cv['cv_splits'][1]['test']['t']: ", len(data_test['cv_splits'][1]['test']['t']))

## ----------- End of Inference Data Preprocessing -----------

# Data for generating embeddings...combine train & test cohorts in one place

### Load Train csv

In [ ]:
# load the csv file
sixmodal_train_data = pd.read_csv('6Modal_Train_Data_mR_Gen_DMeth_Clin_Prot_Mut.csv')
print('Opening file: 6Modal_Train_Data_mR_Gen_DMeth_Clin_Prot_Mut.csv')
print(sixmodal_train_data.shape)
sixmodal_train_data.head()

In [ ]:
print(sixmodal_train_data[(sixmodal_train_data.iloc[:, 3:] == 0).all(axis=1)])

### Load Test csv

In [ ]:
sixmodal_test_data = pd.read_csv('6Modal_Test_Data_mR_Gen_DMeth_Clin_Prot_Mut.csv')
print('Opening file: 6Modal_Test_Data_mR_Gen_DMeth_Clin_Prot_Mut.csv')
print(sixmodal_test_data.shape)
sixmodal_test_data.head()

In [ ]:
print(sixmodal_test_data[(sixmodal_test_data.iloc[:, 3:] == 0).all(axis=1)])

### Combine the Training and Testing dataframes

In [ ]:
# combine sixmodal_train_data and sixmodal_test_data
combined_data = pd.concat([sixmodal_train_data, sixmodal_test_data], ignore_index=True)
print(combined_data.shape)
combined_data.head()

### Divide in 2 folds temporarily

In [ ]:
#get the patient names from sample column
combined_patientnames=combined_data['sample']
print(len(combined_patientnames))
print(combined_patientnames)

In [ ]:
# Generate k-fold cross validation splits for the data, k=10
from sklearn.model_selection import KFold
# initialize the kfold object
kf = KFold(n_splits=2, shuffle=True, random_state=42)
folds = list(kf.split(combined_patientnames))
#save as .csv file having rows as patient names and columns as folds
folds_array = np.zeros((len(combined_patientnames), 2))
for i in range(2):
    folds_array[folds[i][1], i] = 1
folds_df = pd.DataFrame(folds_array, columns=['fold_{}'.format(i) for i in range(1,3)])
folds_df.index = combined_patientnames
#replace 0 with Train and 1 with Test
folds_df = folds_df.replace(0, 'Train')
folds_df = folds_df.replace(1, 'Test')
folds_df.to_csv('pancancer_combined/data_splits.csv')
folds_df

In [ ]:
# save the samples, vital_status (OS) and labels (OS.time) to numpy.ndarray
samples = combined_data['sample'].values
vital_status = combined_data['OS'].values
survival = combined_data['OS.time'].values
print('samples:', samples, 'vital_status:', vital_status, 'labels:', survival)
print('samples shape:', samples.shape, 'vital_status shape:', vital_status.shape, 'survival shape:', survival.shape)
# convert survival to df
survival_df = pd.DataFrame(survival, columns=['Labels'])
survival_df.index = samples
survival_df = survival_df.astype(float)
# convert vital_status to df
vital_status_df = pd.DataFrame(vital_status, columns=['Vital_status'])
vital_status_df.index = samples
vital_status_df = vital_status_df.astype(float)
# remove the vital_status (OS) and labels (OS.time) from the dataframe
print('test shape:', combined_data.shape)
omic = combined_data.drop(columns=['sample', 'OS', 'OS.time'])
omic.index = samples
print('omic shape:', omic.shape)
omic.head()

In [ ]:
 # convert dataframes to numpy arrays
data_cv = {}
data_cv['cv_splits'] = {}

for i in range(2):
    # add and store data for each fold in the data_cv dictionary
    data_cv['cv_splits'][i+1] = {}
    data_cv['cv_splits'][i+1]['test'] = {}
    data_cv['cv_splits'][i+1]['test'] = {
        'x_patname': folds_df[folds_df['fold_{}'.format(i+1)] == 'Test'].index.values.tolist(),
        'x_omic': omic.loc[folds_df[folds_df['fold_{}'.format(i+1)] == 'Test'].index.values.tolist()].values,
        'e': np.squeeze(vital_status_df.loc[folds_df[folds_df['fold_{}'.format(i+1)] == 'Test'].index.values.tolist()].values),
        't': np.squeeze(survival_df.loc[folds_df[folds_df['fold_{}'.format(i+1)] == 'Test'].index.values.tolist()].values)
    }

# save the dictionary to a .pkl file
with open('pancancer_combined/omics_2cv.pkl', 'wb') as f:
    pickle.dump(data_cv, f)

In [ ]:
# VERIFY THE PKL FILE
data_cv = pickle.load(open('pancancer_combined/omics_2cv.pkl', 'rb'))
data_cv_splits = data_cv['cv_splits']

for k, data in data_cv_splits.items():
	print("*******************************************")
	print("************** SPLIT (%d/%d) **************" % (k, len(data_cv_splits.items())))
	print("*******************************************")
	if k == 1:
		print(data_cv_splits)
	print(len(data_cv_splits[k]['test']['x_patname']), (data_cv_splits[k]['test']['e']).shape, (data_cv_splits[k]['test']['t']).shape, (data_cv_splits[k]['test']['x_omic']).shape)

In [ ]:
print('data_cv.keys: ', data_cv.keys())
print("data_cv['cv_splits'].keys: ", data_cv['cv_splits'].keys())
print("data_cv['cv_splits'][1]: ", data_cv['cv_splits'][1].keys())
print("data_cv['cv_splits'][1]['test']: ", data_cv['cv_splits'][1]['test'].keys())

### Now combine the two folds pkl in one test pkl file

In [ ]:
data = data_cv

In [ ]:
print('data.keys: ', data.keys())
for k in range(1, 3):
    print(len(data['cv_splits'][k]['test']['x_patname']), (data['cv_splits'][k]['test']['e']).shape, (data['cv_splits'][k]['test']['t']).shape, (data['cv_splits'][k]['test']['x_omic']).shape)

In [ ]:
combined_test_data = {
            'cv_splits': {
                1: {
                    'test': {'x_patname': [], 'x_omic': [], 'e': [], 't': []} 
                }
            }
        }
print(combined_test_data)
for key in data['cv_splits'].keys():
    print('key: ', key)
    for subkey in data['cv_splits'][key]['test'].keys():
        # print('subkey :', subkey)
        subdata = data['cv_splits'][key]['test'][subkey]
        if np.isscalar(subdata) or np.ndim(subdata) == 0:
            subdata = [subdata]
        combined_test_data['cv_splits'][1]['test'][subkey].extend(subdata)

# Save combined data into a new .pkl file
with open('pancancer_combined/omic_combined.pkl', 'wb') as f:
    print('saving as pancancer_combined/omic_combined.pkl')
    pickle.dump(combined_test_data, f)

### Verify the number of samples in the saved data

In [ ]:
# Load the saved pkl file
data_test = pickle.load(open('pancancer_combined/omic_combined.pkl', 'rb'))
print('opening file: pancancer_combined/omic_combined.pkl')
print('data.keys: ', data_test.keys())
print("data_cv['cv_splits'].keys: ", data_test['cv_splits'].keys())
print("data_cv['cv_splits'][1]: ", data_test['cv_splits'][1].keys())
print("data_cv['cv_splits'][1]['test']: ", data_test['cv_splits'][1]['test'].keys())
print("data_cv['cv_splits'][1]['test']['x_patname']: ", len(data_test['cv_splits'][1]['test']['x_patname']))
print("data_cv['cv_splits'][1]['test']['x_omic']: ", len(data_test['cv_splits'][1]['test']['x_omic']))
print("data_cv['cv_splits'][1]['test']['e']: ", len(data_test['cv_splits'][1]['test']['e']))
print("data_cv['cv_splits'][1]['test']['t']: ", len(data_test['cv_splits'][1]['test']['t']))

# Convert the Embeddings from pkl to parquet

In [ ]:
# load the pkl file having the features for the 33 cancers patients
embdgs = pickle.load(open('pancancer_combined/FINAL_pancancer_combined_omic_embdgs.pkl', 'rb'))
print('opening file: pancancer_combined/FINAL_pancancer_combined_omic_embdgs.pkl')
print('number of patients:', len(embdgs))
print('number of features:', len(embdgs[0]))

In [ ]:
# Get the number of patients and features
num_patients = len(embdgs[0][0])
num_features = len(embdgs[0][1])
print(f"Number of patients: {num_patients}")
print(f"Number of features: {num_features}")
# Get the number of features for each patient
features_per_patient = [len(patient_features) for patient_features in embdgs[0][1]]
print(f"Features per patient: {features_per_patient}")

In [ ]:
# Print the patient name and corresponding feature size
for patient, features in zip(embdgs[0][0], embdgs[0][1]):
    print(f"Patient: {patient}, Feature size: {len(features)}")

In [ ]:
# Create a DataFrame with a column for PatientID and a second column with the embeddings
df = pd.DataFrame({
    'SampleID': embdgs[0][0],
    'Embeddings': embdgs[0][1]
})

# Add a PatientID column as the first column
df.insert(0, 'PatientID', df['SampleID'].str[:-4])

# Save the DataFrame to a parquet file
df.to_parquet("pancancer_combined/patients_embeddings.parquet")

In [ ]:
# Load the parquet file
df = pd.read_parquet("pancancer_combined/patients_embeddings.parquet")
# Print the first few rows of the DataFrame
print(df.head())
# Print the shape of the DataFrame
print(f"Shape: {df.shape}")
# Print the column names
print(f"Columns: {df.columns}")

In [ ]:
df